# **지하철 데이터 전처리**


1~12월달 지하철 인원수 합치기

### **범주별 점수화 함수 정의**
데이터: metro_microdust.json (역별 월평균 미세먼지), weather_seasons.json (계절별 외부 기온).
로직:
미세먼지 점수: 각 역의 연평균 미세먼지 농도를 "좋음(4점) ~ 매우 나쁨(1점)"의 4단계로 점수화합니다.
기온 점수: 현재 계절의 역 주변 평균 기온을 "쾌적(4점) ~ 혹한/혹서(1점)"의 4단계로 점수화합니다.
두 점수를 0~1 사이로 정규화한 후, 쾌적도 = (미세먼지 점수 * 0.57) + (기온 점수 * 0.43) 와 같이 가중 합산하여 최종 점수를 산출합니다.

```
#미세먼지 점수: 각 역의 연평균 미세먼지 농도를 "좋음(4점) ~ 매우 나쁨(1점)"의 4단계로 점수화
def score_microdust_cat(x):
 if x <= 30: return 4
 elif x <= 80: return 3
 elif x <= 150: return 2
 else: return 1

#기온 점수: 현재 계절의 역 주변 평균 기온을 "쾌적(4점) ~ 혹한/혹서(1점)"의 4단계로 점수화
def score_temp_cat(x):
 if 10 <= x <= 24: return 4
 elif (0 <= x < 10) or (24 < x <= 28): return 3
 elif (-10 <= x < 0) or (28 < x <= 34): return 2
 else: return 1
```

3.3. 쾌적성 (evaluate_pleasantness_for_candidates)
데이터: metro_microdust.json (역별 월평균 미세먼지), weather_seasons.json (계절별 외부 기온).
로직:
미세먼지 점수: 각 역의 연평균 미세먼지 농도를 "좋음(4점) ~ 매우 나쁨(1점)"의 4단계로 점수화합니다.
기온 점수: 현재 계절의 역 주변 평균 기온을 "쾌적(4점) ~ 혹한/혹서(1점)"의 4단계로 점수화합니다.
두 점수를 0~1 사이로 정규화한 후, 쾌적도 = (미세먼지 점수 * 0.57) + (기온 점수 * 0.43) 와 같이 가중 합산하여 최종 점수를 산출합니다.

수직 이동 시간(vertical_time): 역의 지상 고도와 에스컬레이터 길이를 바탕으로, 에스컬레이터 이용 구간과 계단/경사로 도보 구간을 분리하여 각각 다른 속도(ES_SPEED, WALK_SPEED)를 적용함으로써 계산의 현실성을 극대화

## **1) 정시성**

In [ ]:
dfs = {}
for fname, content in uploaded.items():
    # ◀— encoding 파라미터 추가
    try:
        df = pd.read_csv(io.BytesIO(content), encoding='cp949')
    except UnicodeDecodeError:
        # 혹시 cp949로 안 읽히면 latin1로 폴백
        df = pd.read_csv(io.BytesIO(content), encoding='latin1')
    print(f"\n=== 파일: {fname} ===")
    print("컬럼:", df.columns.tolist())
    print("레코드 수:", len(df))
    display(df.head())
    dfs[fname] = df

# 3. null 값 제거
accident_df = dfs['metro_accident.csv'].dropna()
late_df     = dfs['metro_late.csv'].dropna()

print("\n■ metro_accident.csv null 제거 후 레코드 수:", len(accident_df))
print("■ metro_late.csv null 제거 후 레코드 수:", len(late_df))


=== 파일: metro_accident.csv ===
컬럼: ['연번', '발생일자', '사고유형', '발생역', '발생시간', '비고']
레코드 수: 2388


,연번,발생일자,사고유형,발생역,발생시간,비고
0,1,2024-12-24,출입문관련,잠실새내역,13:40:00,NaN
1,2,2024-12-21,출입문관련,고속터미널(3)역,11:26:00,NaN
2,3,2024-12-18,출입문관련,장지역,06:26:00,NaN
3,4,2024-12-18,출입문관련,동대문역사문화공원(4)역,15:29:00,NaN
4,5,2024-12-16,출입문관련,강동역,18:46:00,NaN



=== 파일: metro_late.csv ===
컬럼: ['번호', '지연일자', '노선', '지연시간대', '최대지연시간']
레코드 수: 2056


,번호,지연일자,노선,지연시간대,최대지연시간
0,1,2023-09-01,2호선 (외선),첫차~09시,10분
1,2,2023-09-01,2호선 (내선),첫차~09시,5분
2,3,2023-09-01,1호선 (하행선),첫차~09시,15분
3,4,2023-09-01,1호선 (상행선),첫차~09시,15분
4,5,2023-09-01,4호선 (하행선),첫차~09시,10분



■ metro_accident.csv null 제거 후 레코드 수: 0
■ metro_late.csv null 제거 후 레코드 수: 2056


### **1-1) 정시성_ 사고횟수**
지하철 사고 데이터를 담고 있는 metro_accident.csv 파일을 전처리하는 작업에서는
* 파일의 인코딩 방식은 일반적으로 cp949를 사용하되, 만약 인코딩 오류가 발생할 경우를 대비해 latin1 방식으로 대체하여 읽도록 예외처리
* 분석에 필요한 컬럼인 '사고유형'과 '발생역' 두 항목만 추려낸 뒤, 이 중에서도 결측값이 있는 행은 제거하여 실제 분석에 사용할 수 있는 데이터만 선별
* 기존 지하철 역명에서 괄호와 그 안의 내용 제거, '역'이라는 접미사 및 역명의 양끝 공백을 제거하여 정제
* 정제된 지하철 역명은 기존 발생역 칼럼과 별도로 '역명' 이라는 칼럼을 생성하여 저장
* 사고 발생 횟수 집계 시 같은 역에서 여러 건의 사고가 발생한 경우를 하나로 묶어 개수를 세고, 사고 횟수가 많은 역부터 내림차순으로 정렬하여 사고 위험도가 높은 역들을 파악할 수 있도록 정리
* 집계된 사고 횟수 데이터는 json 형식으로 저장하고, 코랩 환경에서 다운로드 할 수 있도록 처리

In [ ]:
# (1) metro_accident.csv를 다시 읽어서 accident_df에 할당
import io, pandas as pd
from google.colab import files

uploaded = files.upload()  # metro_accident.csv 선택
df = pd.read_csv(io.BytesIO(uploaded['metro_accident.csv']), encoding='cp949', on_bad_lines='warn')

# (2) 컬럼명과 전체 행·열 수 출력
print("컬럼명:", df.columns.tolist())
print("전체 크기:", df.shape)
display(df.head())


Saving metro_accident.csv to metro_accident.csv
컬럼명: ['연번', '발생일자', '사고유형', '발생역', '발생시간', '비고']
전체 크기: (2388, 6)


,연번,발생일자,사고유형,발생역,발생시간,비고
0,1,2024-12-24,출입문관련,잠실새내역,13:40:00,NaN
1,2,2024-12-21,출입문관련,고속터미널(3)역,11:26:00,NaN
2,3,2024-12-18,출입문관련,장지역,06:26:00,NaN
3,4,2024-12-18,출입문관련,동대문역사문화공원(4)역,15:29:00,NaN
4,5,2024-12-16,출입문관련,강동역,18:46:00,NaN


In [ ]:
print(df[['사고유형','발생역']].isnull().sum())


사고유형    0
발생역     0
dtype: int64


In [ ]:
acc_df = df[['사고유형','발생역']].dropna(subset=['사고유형','발생역'])
print("필터 후 크기:", acc_df.shape)
display(acc_df.head())


필터 후 크기: (2388, 2)


,사고유형,발생역
0,출입문관련,잠실새내역
1,출입문관련,고속터미널(3)역
2,출입문관련,장지역
3,출입문관련,동대문역사문화공원(4)역
4,출입문관련,강동역


In [ ]:
# 고유한 역 이름 몇 개만 보기
print("역 이름 샘플:", acc_df['발생역'].unique()[:10])


역 이름 샘플: ['잠실새내역' '고속터미널(3)역' '장지역' '동대문역사문화공원(4)역' '강동역' '철산역' '잠원역' '자양역'
 '합정(6)역' '청담역']


In [ ]:
import re

# 1) 공백·특수문자 정리용 함수 정의
def clean_station(name):
    # 1. 괄호( ) 와 그 안의 내용 제거
    name = re.sub(r'\(.*?\)', '', name)
    # 2. 끝에 붙은 '역' 제거
    name = re.sub(r'역$', '', name)
    # 3. 양끝 공백 제거
    return name.strip()

In [ ]:
# 2) 새 컬럼으로 저장 (원본 덮어쓰셔도 됩니다)
acc_df['역명'] = acc_df['발생역'].apply(clean_station)

# 3) 확인
from IPython.display import display
print("정제된 역 이름 샘플:", acc_df['역명'].unique()[:10])
display(acc_df[['발생역', '역명']].head(10))

# 4) 이제 이 컬럼으로 그룹화
station_accident_counts = (
    acc_df
    .groupby('역명')
    .size()
    .reset_index(name='사고횟수')
    .sort_values('사고횟수', ascending=False)
)

display(station_accident_counts)

정제된 역 이름 샘플: ['잠실새내' '고속터미널' '장지' '동대문역사문화공원' '강동' '철산' '잠원' '자양' '합정' '청담']


,발생역,역명
0,잠실새내역,잠실새내
1,고속터미널(3)역,고속터미널
2,장지역,장지
3,동대문역사문화공원(4)역,동대문역사문화공원
4,강동역,강동
5,철산역,철산
6,잠원역,잠원
7,자양역,자양
8,합정(6)역,합정
9,청담역,청담


,역명,사고횟수
80,동대문역사문화공원,99
130,사당,60
169,신도림,49
21,고속터미널,47
29,교대,44
...,...,...
291,화서,1
292,화정,1
0,3130열차,1
295,효창공원,1


In [ ]:
# 1) DataFrame을 records 형태의 JSON으로 저장
station_accident_counts.to_json(
    'station_accident_counts.json',
    orient='records',
    force_ascii=False,
    indent=2
)

# 2) Colab에서 파일 다운로드
from google.colab import files
files.download('station_accident_counts.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **1-2) 정시성_ 연착시간**
지하철 열차 지연 데이터를 담고 있는 metro_late.csv 파일을 전처리하는 작업에서는
* 파일의 인코딩 방식은 일반적으로 cp949를 사용하되, 만약 인코딩 오류가 발생할 경우를 대비해 latin1 방식으로 대체하여 읽도록 예외처리
* ['번호', '지연일자', '노선', '지연시간대', '최대지연시간'] 칼럼 중 '번호' 칼럼 제거
* '노선' 칼럼에서는 정규표현식을 이용하여 괄호 안의 내용 및 문자열 양끝의 공백을 제거
* '최대지연시간'이라는 칼럼명은 명확성을 위해 '최대지연시간(분)'으로 수정하고, 문자열 형태의 값들로부터 숫자만 추출하여 정수형(int)로 변환
* 집계된 연착률(지연시간) 데이터는 json 형식으로 저장하고, 코랩 환경에서 다운로드 할 수 있도록 처리

In [ ]:
from google.colab import files
import io
import pandas as pd
import re
from IPython.display import display

# 1. 파일 업로드
uploaded = files.upload()
fname = list(uploaded.keys())[0]

# 2. CSV 읽기 (CP949 우선, 실패 시 Latin-1 폴백)
try:
    df = pd.read_csv(io.BytesIO(uploaded[fname]), encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(io.BytesIO(uploaded[fname]), encoding='latin1')

# 3. 컬럼 목록·샘플 확인
print("▶ 파일:", fname)
print("컬럼:", df.columns.tolist())
print("레코드 수:", df.shape[0])
display(df.head())


Saving metro_late.csv to metro_late.csv
▶ 파일: metro_late.csv
컬럼: ['번호', '지연일자', '노선', '지연시간대', '최대지연시간']
레코드 수: 2056


,번호,지연일자,노선,지연시간대,최대지연시간
0,1,2023-09-01,2호선 (외선),첫차~09시,10분
1,2,2023-09-01,2호선 (내선),첫차~09시,5분
2,3,2023-09-01,1호선 (하행선),첫차~09시,15분
3,4,2023-09-01,1호선 (상행선),첫차~09시,15분
4,5,2023-09-01,4호선 (하행선),첫차~09시,10분


In [ ]:
# 2. '번호' 컬럼 제거
df = df.drop(columns=['번호'])

# 3. '노선' 컬럼에서 괄호 내용 제거
df['노선'] = df['노선'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

# 4. '최대지연시간' → '최대지연시간(분)'으로 컬럼명 변경
df = df.rename(columns={'최대지연시간': '최대지연시간(분)'})

# 5. '최대지연시간(분)' 값에서 숫자만 추출하여 정수형으로 변환
df['최대지연시간(분)'] = (
    df['최대지연시간(분)']
    .astype(str)
    .str.extract(r'(\d+)')[0]
    .astype(int)
)

# 6. 결과 확인
print("■ 전처리 후 컬럼:", df.columns.tolist())
display(df.head())

■ 전처리 후 컬럼: ['지연일자', '노선', '지연시간대', '최대지연시간(분)']


,지연일자,노선,지연시간대,최대지연시간(분)
0,2023-09-01,2호선,첫차~09시,10
1,2023-09-01,2호선,첫차~09시,5
2,2023-09-01,1호선,첫차~09시,15
3,2023-09-01,1호선,첫차~09시,15
4,2023-09-01,4호선,첫차~09시,10


In [ ]:
# 7. JSON으로 저장 및 다운로드
df.to_json(
    'metro_late_cleaned.json',
    orient='records',
    force_ascii=False,
    indent=2
)
files.download('metro_late_cleaned.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 함수입히기
* 서로 다른 문자 인코딩 방식으로 저장된 CSV 파일들을 안정적으로 읽어오기 위해 utf-8, euc-kr, cp949, latin1 순으로 여러 인코딩 방식을 통하여 파일이 정상적으로 읽힐 때까지 차례로 테스트


In [ ]:
import os

In [ ]:
# 3. 다중 인코딩 시도 함수 정의
def read_csv_with_encodings(filepath, encodings=['utf-8', 'euc-kr', 'cp949', 'latin1']):
    for enc in encodings:
        try:
            df = pd.read_csv(filepath, encoding=enc)
            print(f"✅ {filepath} 읽기 성공 (encoding={enc})")
            return df
        except Exception as e:
            print(f"❌ {filepath} - encoding={enc} 실패: {e}")
    raise Exception(f"{filepath}를 지정한 인코딩으로 읽을 수 없습니다.")

# 4. 파일 존재 여부에 따라 읽기
dataframes = {}
for fname in ['metro_transfer_time.csv', 'metro_people.csv', 'confuse.csv']:
    if os.path.exists(fname):
        dataframes[fname] = read_csv_with_encodings(fname)
    else:
        print(f"⚠️ {fname} 파일을 찾을 수 없습니다.")

# 5. JSON 파일 읽기
for jname in ['metro_accident_counts.json', 'metro_late.json']:
    if os.path.exists(jname):
        dataframes[jname] = pd.read_json(jname)
        print(f"✅ {jname} 읽기 성공 (JSON)")
    else:
        print(f"⚠️ {jname} 파일을 찾을 수 없습니다.")

# 6. DataFrame 미리보기
for fname, df in dataframes.items():
    print(f"\n=== {fname} (shape={df.shape}) ===")
    display(df.head())
    display(df.tail())

❌ metro_transfer_time.csv - encoding=utf-8 실패: 'utf-8' codec can't decode byte 0xbf in position 0: invalid start byte
✅ metro_transfer_time.csv 읽기 성공 (encoding=euc-kr)
❌ metro_people.csv - encoding=utf-8 실패: 'utf-8' codec can't decode byte 0xbf in position 0: invalid start byte
✅ metro_people.csv 읽기 성공 (encoding=euc-kr)
⚠️ confuse.csv 파일을 찾을 수 없습니다.
✅ metro_accident_counts.json 읽기 성공 (JSON)
✅ metro_late.json 읽기 성공 (JSON)

=== metro_transfer_time.csv (shape=(140, 6)) ===


,연번,호선,환승역명,환승노선,환승거리,환승소요시간
0,1,1,서울역,4호선,159,02:13
1,2,1,서울역,공항철도,309,04:18
2,3,1,서울역,경의중앙선,164,02:17
3,4,1,시청,2호선,101,01:24
4,5,1,종로3가,3호선,118,01:38


,연번,호선,환승역명,환승노선,환승거리,환승소요시간
135,136,8,석촌,9호선,82,01:08
136,137,8,잠실,2호선,190,02:38
137,138,8,가락시장,3호선,35,00:29
138,139,8,복정,수인분당선,16,00:13
139,140,8,모란,수인분당선,99,01:23



=== metro_people.csv (shape=(199270, 26)) ===


,연번,수송일자,호선,역번호,역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,1,2023-01-01,1호선,150,서울역,승차,215,145,231,594,...,2655,2509,2696,2549,2462,2177,2190,1808,734,7
1,2,2023-01-01,1호선,150,서울역,하차,154,636,595,939,...,2282,2295,2526,1930,1897,1487,991,609,280,46
2,3,2023-01-01,1호선,151,시청,승차,48,73,106,194,...,843,895,959,985,670,630,515,330,146,0
3,4,2023-01-01,1호선,151,시청,하차,64,247,293,463,...,602,575,533,456,285,267,246,154,79,18
4,5,2023-01-01,1호선,152,종각,승차,407,235,158,201,...,1145,1402,1223,1272,911,913,906,602,232,3


...

,역명,사고횟수
0,동대문역사문화공원,99
1,사당,60
2,신도림,49
3,고속터미널,47
4,교대,44


,역명,사고횟수
292,화서,1
293,화정,1
294,3130열차,1
295,효창공원,1
296,효창공원앞,1



=== metro_late.json (shape=(2056, 4)) ===


,지연일자,노선,지연시간대,최대지연시간(분)
0,2023-09-01,2호선,첫차~09시,10
1,2023-09-01,2호선,첫차~09시,5
2,2023-09-01,1호선,첫차~09시,15
3,2023-09-01,1호선,첫차~09시,15
4,2023-09-01,4호선,첫차~09시,10


,지연일자,노선,지연시간대,최대지연시간(분)
2051,2024-08-30,4호선,첫차~09시,10
2052,2024-08-30,4호선,첫차~09시,10
2053,2024-08-30,3호선,첫차~09시,10
2054,2024-08-30,3호선,첫차~09시,5
2055,2024-08-30,2호선,첫차~09시,10


## **3) 환승편의성_ 지하철 별 역사 내 정보**

# 새 섹션

### **리프트**
지하철 역사 내 리프트 갯수 데이터를 담고 있는 metro_lift.json 파일을 전처리하는 작업에서는
* json 파일 안에 있는 여러 계층으로 이뤄져있는 데이터를 데이터프레임 형태로 변환
* 칼럼 ['sbwy_stn_cd','sgg_cd','emd_nm','emd_cd','node_wkt','sgg_nm','node_type','node_id','sbwy_stn_nm','node_type_cd'] 중 불필요한 칼럼     'sbwy_stn_cd','sgg_cd','emd_nm','emd_cd','node_wkt','sgg_nm','node_type','node_id'은 삭제하고, 주요 분석 대상이 되는 컬럼인 'sbwy_stn_nm'은 ‘역명’으로, 'node_type_cd'는 ‘lift_count’로 변경하여 보기 쉽게 변경
* 리프트의 갯수 데이터를 담고 있는 'lift_count'칼럼을 정수형 데이터로 변환하고, 오류가 나는 항목은 NaN 처리한 후 0으로 대체
* 최족적으로는 ‘역명’을 기준으로 그룹화하여 각 역에 설치된 리프트 수를 모두 합산




In [ ]:
from google.colab import files
import io, json
import pandas as pd

# 1) 파일 업로드
uploaded = files.upload()
fname = list(uploaded.keys())[0]

# 2) JSON 로드
with io.BytesIO(uploaded[fname]) as bio:
    data = json.load(bio)
records = data.get('DATA', data)

# 3) DataFrame 변환
df = pd.json_normalize(records)

# 4) 컬럼 리네임 & 드롭
df = df.rename(
    columns={
        'sbwy_stn_nm': '역명',
        'node_type_cd': 'lift_count'
    }
).drop(columns=[
    'sbwy_stn_cd','sgg_cd','emd_nm','emd_cd',
    'node_wkt','sgg_nm','node_type','node_id'
])

# 1) lift_count 컬럼을 정수형으로 변환
df['lift_count'] = pd.to_numeric(df['lift_count'], errors='coerce').fillna(0).astype(int)

# 2) 역명별 lift_count 합계
station_lift = (
    df
    .groupby('역명', as_index=False)['lift_count']
    .sum()
    .sort_values('lift_count', ascending=False)
)

display(station_lift)


Saving metro_lift.json to metro_lift (1).json


,역명,lift_count
38,종로3가,6
17,동묘앞,5
11,대청,4
27,수락산,4
12,대치,4
35,일원,4
33,월계,3
14,도곡,3
19,마천,2
6,남구로,2


### **엘레베이터**
지하철 역사 별 엘리베이터 갯수 데이터를 담고 있는 metro_elevator.json 파일을 전처리하는 작업에서는
* json 파일 안에 있는 여러 계층으로 이뤄져있는 데이터를 데이터프레임 형태로 변환
* 칼럼 ['sbwy_stn_nm','sbwy_stn_cd', 'sgg_cd', 'emd_nm', 'emd_cd','node_wkt', 'sgg_nm', 'node_type', 'node_id''node_type_cd'] 중 불필요한 칼럼 'sbwy_stn_cd', 'sgg_cd', 'emd_nm', 'emd_cd','node_wkt', 'sgg_nm', 'node_type', 'node_id'은 삭제하고, 주요 분석 대상이 되는 컬럼인 'sbwy_stn_nm'은 ‘역명’으로, 'node_type_cd'는 ‘elevator_count’로 변경하여 보기 쉽게 변경
* 리프트의 갯수 데이터를 담고 있는 'elevator_count'칼럼을 정수형 데이터로 변환하고, 오류가 나는 항목은 NaN 처리한 후 0으로 대체
* 최종적으로는 ‘역명’을 기준으로 설치된 엘리베이터의 총합을 집계함

In [ ]:
from google.colab import files
import io, json
import pandas as pd
from IPython.display import display

# 1) metro_elevator.json 파일 업로드
print("▶ metro_elevator.json을 업로드하세요.")
uploaded = files.upload()

# 2) 업로드된 파일 중 metro_elevator.json 읽기
elev_fname = next(name for name in uploaded if name.endswith('metro_elevator.json'))
data = json.loads(uploaded[elev_fname].decode('utf-8')).get('DATA', [])

# 3) DataFrame 변환
df_elev = pd.json_normalize(data)

# 4) 컬럼명 변경 및 불필요 컬럼 제거
df_elev = (
    df_elev
    .rename(columns={'sbwy_stn_nm': '역명', 'node_type_cd': 'elevator_count'})
    .drop(columns=['sbwy_stn_cd', 'sgg_cd', 'emd_nm', 'emd_cd',
                   'node_wkt', 'sgg_nm', 'node_type', 'node_id'])
)

# 5) elevator_count 정수형 변환
df_elev['elevator_count'] = pd.to_numeric(df_elev['elevator_count'], errors='coerce').fillna(0).astype(int)

# 6) 역명별 elevator_count 합계 집계
station_elevator = (
    df_elev
    .groupby('역명', as_index=False)['elevator_count']
    .sum()
    .sort_values('elevator_count', ascending=False)
)

# 7) 결과 확인
display(station_elevator.head(20))


▶ metro_elevator.json을 업로드하세요.


Saving metro_elevator.json to metro_elevator.json


,역명,elevator_count
202,온수(성공회대입구),7
53,노원,6
134,서울역,6
116,삼각지,6
161,신림,5
102,방학,5
59,당산,4
218,이촌(국립중앙박물관),4
7,강남구청,4
117,삼성(무역센터),4


### **화장실**
지하철 역사 별 화장실 데이터를 담고 있는 metro_restroom.csv 파일을 전처리하는 작업에서는
* csv 파일 'metro_restroom.csv'를 cp949 인코딩으로 읽어옴
* 칼럼 ['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ', '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호', '상세위치 ', '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ', '남성용-어린이용소변기수 ', '여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ', '비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ', '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ', '기저귀교환대설치유무-여자장애인화장실 ', '리모델링 연도', '데이터기준일자'] 중에서 ['운영노선명 ', '역명', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '여성용-대변기수 ', '화장실입구cctv설치유무 ', '리모델링 연도'] 만을 추출
* 추출한 칼럼들의 칼럼명을 편의성을 위해 '운영노선명 ' 은 'operated_line'으로, '역명' 은 'station_name'으로, '남녀공용화장실여부 ' 은 'unisex'으로, '남성용-대변기수 ' 는 'male_toilet_count'으로, '남성용-소변기수 '은 'male_urinal_count'으로, '여성용-대변기수 ' 은 'female_toilet_count'으로, '화장실입구cctv설치유무 ' 은 'cctv_entrance'으로, '리모델링 연도' 은 'remodel_year'으로 이름 재지정
* 'station_name' 컬럼의 좌우 공백 제거 및 줄바꿈·탭 등의 공백문자 제거
* 화장실 기기 개수 관련 컬럼인 'male_toilet_count', 'male_urinal_count', 'female_toilet_count'을 정수형으로 변환하고, 변환 오류 또는 결측치는 0으로 대체
* 'remodel_year' 컬럼은 정수형(Int64)으로 변환하되, 결측값은 그대로 유지함

---

지하철 역사 별 화장실 데이터를 담고 있는 metro_inside_data.csv 파일을 전처리하는 작업에서는
* 지하철 역 내부 설비 데이터 전처리 (metro_inside_data.csv)
csv 파일 'metro_inside_data.csv'를 cp949 인코딩으로 읽어옴
*  칼럼 ['노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT', '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명', '읍면동코드', '읍면동명', '지하철역코드', '지하철역명', '리프트', '엘리베이터'] 중 '지하철역명' 컬럼을 'station_name' 컬럼으로 변환하고 문자열로 처리한 후 공백 제거하고, '지하철역코드' 컬럼도 'station_cd' 컬럼으로 변환하고 문자열로 처리한 후 공백 제거

---
지하철역 및 노선 메타정보인 data-metro-station-1.0.0.json과 data-metro-line-1.0.0.json를 전처리하는 작업에서는

* json 파일 'data-metro-station-1.0.0.json'에서 'DATA' 항목만 추출하여 pandas의 json_normalize 함수를 사용해 데이터프레임으로 변환
* json 파일 'data-metro-line-1.0.0.json'에서 각 노선의 연결 정보가 포함된 리스트 'node'를 record_path로 지정하고, 노선 메타정보인 'line', 'line_name', 'line_subname', 'color'를 meta 인자로 포함하여 데이터프레임으로 변환
* json_normalize를 통해 각 노선 및 역 간 연결 구조를 테이블 형태로 평탄화하여 추출




In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# 2-2) metro_restroom.csv 읽기
restroom_df = pd.read_csv('metro_restroom.csv', encoding='cp949', low_memory=False)
print("---- 1) metro_restroom.csv (화장실 정보) ----")
print("Shape:", restroom_df.shape)
print("Columns:", restroom_df.columns.tolist())
print("상위 5개 행:\n", restroom_df.head(), "\n")

# ──────────────────────────────────────────────────────────────────────────
# 2-3) metro_inside_data.csv 읽기
inside_df = pd.read_csv('metro_inside_data.csv', encoding='cp949', low_memory=False)
print("---- 2) metro_inside_data.csv (역 내부 데이터) ----")
print("Shape:", inside_df.shape)
print("Columns:", inside_df.columns.tolist())
print("상위 5개 행:\n", inside_df.head(), "\n")


---- 1) metro_restroom.csv (화장실 정보) ----
Shape: (312, 36)
Columns: ['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ', '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호', '상세위치 ', '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ', '남성용-어린이용소변기수 ', '여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ', '비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ', '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ', '기저귀교환대설치유무-여자장애인화장실 ', '리모델링 연도', '데이터기준일자']
상위 5개 행:
    연번     구분    관리기관명 운영노선명     역명                   소재지도로명주소   \
0  1.0  공중화장실  서울교통공사    1호선    서울  서울특별시 중구 세종대로 지하2(남대문로 5가)   
1  2.0  공중화장실  서울교통공사    1호선    시청     서울특별시 중구 세종대로 지하101(정동)   
2  3.0  공중화장실  서울교통공사    1호선    종각     서울특별시 종로구 종로 지하55(종로1가)   
3  4.0  공중화장실  서울교통공사    1호선    종각     서울특별시 종로구 종로 지하55(종로1가)   
4  5.0  공중화장실  서울교통공사    1호선  종로3가    서울특별시 종로구 종로 지하129(종로3가)   

                         소재지지번주소         위도    

In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# 2-4) data-metro-station-1.0.0.json 읽기 (역별 기본 정보)
with open('/content/data-metro-station-1.0.0.json', 'r', encoding='utf-8') as f:
    station_json = json.load(f)

station_df = pd.json_normalize(station_json['DATA'])
print("---- 3) data-metro-station-1.0.0.json (지하철 역 기본 정보) ----")
print("Shape:", station_df.shape)
print("Columns:", station_df.columns.tolist())
print("상위 5개 행:\n", station_df.head(), "\n")

# ──────────────────────────────────────────────────────────────────────────
# 2-5) data-metro-line-1.0.0.json 읽기 (호선별 역간 연결 정보)
with open('/content/data-metro-line-1.0.0.json', 'r', encoding='utf-8') as f:
    line_json = json.load(f)

# record_path='node' 아래에 역간 연결(출발역↔도착역) 정보가 list 형태로 들어있습니다.
line_df = pd.json_normalize(
    line_json['DATA'],
    record_path=['node'],
    meta=['line', 'line_name', 'line_subname', 'color'],
    errors='ignore'
)
print("---- 4) data-metro-line-1.0.0.json (지하철 노드 연결) ----")
print("Shape:", line_df.shape)
print("Columns:", line_df.columns.tolist())
print("상위 5개 행:\n", line_df.head(), "\n")

---- 3) data-metro-station-1.0.0.json (지하철 역 기본 정보) ----
Shape: (785, 10)
Columns: ['line', 'name', 'station_nm_chn', 'station_nm_jpn', 'station_nm_eng', 'fr_code', 'station_cd', 'bldn_id', 'lat', 'lng']
상위 5개 행:
   line name station_nm_chn station_nm_jpn station_nm_eng fr_code station_cd  \
0  1호선  망월사            望月寺         マンウォルサ      Mangwolsa     112       1904   
1  1호선  신도림            新道林          シンドリム       Sindorim     140       1007   
2  1호선   금정             衿井          クムジョン      Geumjeong    P149       1708   
3  1호선   보산             保山            ポサン          Bosan     102       1914   
4  1호선   덕정             德亭          トクチョン      Deokjeong     105       1911   

  bldn_id        lat         lng  
0    1904  37.709914  127.047455  
1    1007  37.508787  126.891144  
2    1708  37.372221  126.943429  
3    1914  37.913702  127.057277  
4    1911  37.843188  127.061277   

---- 4) data-metro-line-1.0.0.json (지하철 노드 연결) ----
Shape: (763, 6)
Columns: ['station', 'via', 'li

In [ ]:
# 1-1) metro_restroom.csv 읽기
#    Colab에서 /content/metro_restroom.csv 경로를 확인하고 필요시 경로를 수정하세요.
restroom_df = pd.read_csv('/content/metro_restroom.csv', encoding='cp949', low_memory=False)
print("원본 restroom_df Shape:", restroom_df.shape)
print("원본 restroom_df Columns:\n", restroom_df.columns.tolist(), "\n")


원본 restroom_df Shape: (312, 36)
원본 restroom_df Columns:
 ['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ', '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호', '상세위치 ', '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ', '남성용-어린이용소변기수 ', '여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ', '비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ', '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ', '기저귀교환대설치유무-여자장애인화장실 ', '리모델링 연도', '데이터기준일자'] 



In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# 2) 필요한 컬럼만 추출하기
# 아래 리스트는 사용자가 필요하다고 명시한 컬럼명(원본 한글 컬럼명)입니다.
need_cols = [
    '운영노선명 ',          # 호선 정보
    '역명',                 # 이걸 기준으로 매핑할 것
    '남녀공용화장실여부 ',   # 공용 여부
    '남성용-대변기수 ',      # 남자 대변기 수
    '남성용-소변기수 ',      # 남자 소변기 수
    '여성용-대변기수 ',
    '화장실입구cctv설치유무 ',
    '리모델링 연도'
]

# 실제로 CSV에 이 컬럼들이 모두 존재하는지 확인
print("존재하지 않는 컬럼이 있으면 에러가 납니다:")
for c in need_cols:
    if c not in restroom_df.columns:
        print("   ❗ 해당 컬럼이 없습니다:", repr(c))

존재하지 않는 컬럼이 있으면 에러가 납니다:


In [ ]:
restroom_small = restroom_df[need_cols].copy()
print("\n필요 컬럼만 추출한 restroom_small Shape:", restroom_small.shape)
print("추출 후 Columns:\n", restroom_small.columns.tolist(), "\n")



필요 컬럼만 추출한 restroom_small Shape: (312, 8)
추출 후 Columns:
 ['운영노선명 ', '역명', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '여성용-대변기수 ', '화장실입구cctv설치유무 ', '리모델링 연도'] 



In [ ]:
restroom_small = restroom_small.rename(columns={
    '운영노선명 ': 'operated_line',
    '역명': 'station_name',
    '리모델링 연도': 'remodel_year', # 쾌적도
    '남녀공용화장실여부 ': 'unisex', # 쾌적도
    '화장실입구cctv설치유유무 ': 'cctv_entrance',
    '남성용-대변기수 ': 'male_toilet_count',
    '남성용-소변기수 ': 'male_urinal_count',
    '여성용-대변기수 ': 'female_toilet_count'
})

print("영어 컬럼명으로 변경 후 Columns:\n", restroom_small.columns.tolist(), "\n")


영어 컬럼명으로 변경 후 Columns:
 ['operated_line', 'station_name', 'unisex', 'male_toilet_count', 'male_urinal_count', 'female_toilet_count', '화장실입구cctv설치유무 ', 'remodel_year'] 



In [ ]:
 #──────────────────────────────────────────────────────────────────────────
# 2) station_name(역명) 공백 제거
#    2-1) 앞뒤 일반 공백 strip
restroom_small['station_name'] = restroom_small['station_name'].astype(str).str.strip()

#    2-2) 혹시 모를 탭/줄바꿈 등 모든 공백문자 제거 (내부 띄어쓰기는 보존되지 않을 수 있으므로
#         “역명 자체에 띄어쓰기가 없다고 가정”하거나, 따로 확인이 필요합니다.)
restroom_small['station_name'] = restroom_small['station_name'].str.replace(r'\s+', ' ', regex=True).str.strip()

print("공백 제거 후 station_name 예시:\n", restroom_small['station_name'].unique()[:10], "\n")


공백 제거 후 station_name 예시:
 ['서울' '시청' '종각' '종로3가' '종로5가' '동대문' '동묘앞' '신설동' '제기동' '청량리'] 



In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# 4) 숫자 컬럼(화장실 개수 등) 타입 변환 및 결측 처리
restroom_small['male_toilet_count'] = pd.to_numeric(restroom_small['male_toilet_count'], errors='coerce').fillna(0).astype(int)
restroom_small['male_urinal_count']  = pd.to_numeric(restroom_small['male_urinal_count'], errors='coerce').fillna(0).astype(int)
restroom_small['female_toilet_count'] = pd.to_numeric(restroom_small['female_toilet_count'], errors='coerce').fillna(0).astype(int)
restroom_small['remodel_year']       = pd.to_numeric(restroom_small['remodel_year'], errors='coerce').astype('Int64')  # 결측일 수 있음
# unisex, cctv_entrance는 원래 Y/N이라면 아래처럼 매핑해서 True/False로 변환도 가능
# restroom_small['unisex']        = restroom_small['unisex'].map({'Y': True, 'N': False})
# restroom_small['cctv_entrance'] = restroom_small['cctv_entrance'].map({'Y': True, 'N': False})

# ──────────────────────────────────────────────────────────────────────────
# 5) 전처리 완료된 결과 확인
print("전처리 완료된 restroom_small Shape:", restroom_small.shape)
print("상위 10개 행:\n", restroom_small.head(10), "\n")
print("컬럼별 데이터 타입:\n", restroom_small.dtypes)

전처리 완료된 restroom_small Shape: (312, 8)
상위 10개 행:
   operated_line station_name unisex  male_toilet_count  male_urinal_count  \
0           1호선           서울      N                  4                  5   
1           1호선           시청      N                  6                  5   
2           1호선           종각      N                  3                  2   
3           1호선           종각      N                  5                  5   
4           1호선         종로3가      N                  5                  6   
5           1호선         종로3가      N                  4                  5   
6           1호선         종로5가      N                  4                  4   
7           1호선          동대문      N                  4                  6   
8           1호선          동묘앞      N                  1                  2   
9           1호선          동묘앞      N                  1                  2   

   female_toilet_count 화장실입구cctv설치유무   remodel_year  
0                    9              Y          2

In [ ]:
# 1-2) metro_inside_data.csv 읽기
inside_df = pd.read_csv('/content/metro_inside_data.csv', encoding='cp949', low_memory=False)
print("원본 inside_df Shape:", inside_df.shape)
print("원본 inside_df Columns:\n", inside_df.columns.tolist(), "\n")

원본 inside_df Shape: (6563, 18)
원본 inside_df Columns:
 ['노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT', '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명', '읍면동코드', '읍면동명', '지하철역코드', '지하철역명', '리프트', '엘리베이터'] 



In [ ]:
# 1-2-1) station_name, station_cd, 노드/링크, 엘리베이터·리프트 정보 등
inside_df['station_name'] = inside_df['지하철역명'].astype(str).str.strip()
inside_df['station_cd']   = inside_df['지하철역코드'].astype(str).str.strip()

In [ ]:
['노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT', '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명', '읍면동코드', '읍면동명', '지하철역코드', '지하철역명', '리프트', '엘리베이터']

### **리프트, 엘레베이터, 에스컬레이터, 화장실 합치기**

metro_els.json • metro_restroom.csv • metro_ele_lift_count.json 전처리 과정은 다음과 같다
* metro_ele_lift_count.json 파일, metro_restroom.csv 파일, metro_els.json 파일 부르기
* metro_ele_lift_count.json의 경우,
    * 원본 칼럼 ['역명', '리프트', '엘리베이터']을 ['station_name','lift_count','elevator_count'] 로 변경
* metro_restroom.csv 의 경우,
    * 원본 칼럼 ['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ', '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호', '상세위치 ', '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ', '남성용-어린이용소변기수 ', '여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ', '비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ', '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ', '기저귀교환대설치유무-여자장애인화장실 ', '리모델링 연도', '데이터기준일자'] 중에서 ['운영노선명 ', '역명', '게이트 내외 구분', '개방시간 ', '화장실입구cctv설치유무 ', '리모델링 연도']만 추출
    * 추출한 6개의 칼럼을 영어로 칼럼명 재지정 ['line','station_name' 'gate_access','open_hours','cctv_at_entrance','remodel_year']
* metro_els.json 의 경우,
    * 원본 칼럼 ['elvtr_no', 'eqpmnt', 'opr_drct', 'sbwy_rout_ln', 'instl_hgt', 'no', 'opr_len', 'plf_pbadms', 'opr_sec', 'sbwy_stns_nm'] 의 칼럼명 중 다음 칼럼들은 영어로 재지정 ['opr_drct': 'direction', 'sbwy_rout_ln': 'line', 'instl_hgt': 'installation_height', 'no': 'unit_no', 'opr_len': 'escalator_length', 'opr_sec': 'section', 'sbwy_stns_nm': 'station_name']
    * 그 중 ['direction', 'line', 'section', 'unit_no', 'installation_height', 'escalator_length', 'station_name']만 추출


---
combined_facilities.json를 형성하게 된 과정은 다음과 같다.
* metro_els.json • metro_restroom.csv • metro_ele_lift_count.json 합치기 (리프트, 엘리베이터, 에스컬레이터, 화장실 합치기)
    * metro_ele_lift_count.json의 경우
        * 원본 칼럼 ['역명', '리프트', '엘리베이터']에서 ['역명', '엘리베이터', '리프트'] 순서로 추출
        * 추출한 칼럼명을 ['station_name', 'elevator_count', 'lift_count']
로 변경

    * metro_restroom.csv의 경우
        * 원본 칼럼 ['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ', '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호','상세위치 ', '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ', '남성용-어린이용소변기수 ','여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ','비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ', '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ','기저귀교환대설치유무-여자장애인화장실 ', '리모델링 연도', '데이터기준일자'] 중에서['운영노선명 ', '역명', '게이트 내외 구분', '개방시간 ', '화장실입구cctv설치유무 ', '리모델링 연도']만 추출
        * 추출한 6개 칼럼명을 ['line', 'station_name', 'gate_access', 'open_hours', 'cctv_at_entrance', 'remodel_year']와 같이 영어로 재지정
    * metro_els.json의 경우
        * 원본 칼럼 ['elvtr_no', 'eqpmnt', 'opr_drct', 'sbwy_rout_ln', 'instl_hgt', 'no', 'opr_len', 'plf_pbadms', 'opr_sec', 'sbwy_stns_nm']중 ['opr_drct', 'sbwy_rout_ln', 'opr_sec', 'no', 'instl_hgt', 'opr_len', 'sbwy_stns_nm'] 7개의 칼럼만 추출
        * 추출된 칼럼명을 ['direction', 'line', 'section', 'unit_no', 'installation_height', 'escalator_length', 'station_name']로 변경
* ele_df와 rest_df를 outer 방식으로 병합한 후, 그 결과에 esk_df를 outer 방식으로 병합
* 병합된 결과의 칼럼은 ['station_name', 'lift_count', 'elevator_count',
'line', 'gate_access', 'open_hours', 'cctv_at_entrance', 'remodel_year', 'direction', 'section', 'unit_no', 'installation_height', 'escalator_length']의 순서로 재배열
* 최종 결과를 combined_facilities.json으로 저장

In [ ]:
# 2) Load each into a DataFrame and display columns
dataframes = {}

# Elevator & lift counts
ele_file = next(fn for fn in uploaded if 'lift' in fn or 'ele' in fn)
dataframes['ele_lift'] = pd.read_json(ele_file)
print(f"Columns in {ele_file}: {dataframes['ele_lift'].columns.tolist()}")
display(dataframes['ele_lift'].head())
ele_df = pd.read_json(ele_file)


Columns in metro_ele_lift_count.json: ['역명', '리프트', '엘리베이터']


,역명,리프트,엘리베이터
0,종로3가,6,3
1,동묘앞,5,2
2,수락산,4,2
3,일원,4,2
4,대청,4,2


In [ ]:
import pandas as pd
import re
uploaded = files.upload()

# 1. CSV 파일 읽기 (CP949 우선, 실패 시 Latin1)
try:
    df_inside = pd.read_csv('metro_inside_data.csv', encoding='cp949')
except UnicodeDecodeError:
    df_inside = pd.read_csv('metro_inside_data.csv', encoding='latin1')

# 2. 불필요 컬럼 제거
drop_cols = [
    '노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드',
    '링크 WKT', '링크 ID', '링크 유형 코드',
    '시작노드 ID', '종료노드 ID', '링크 길이',
    '시군구코드', '시군구명', '읍면동코드', '읍면동명', '지하철역코드'
]
df_clean = df_inside.drop(columns=drop_cols)

# 3. 컬럼명 변경 및 역명 정제
df_clean = df_clean.rename(columns={'지하철역명': '역명'})

def clean_station(name):
    # ( ) 제거 및 끝 '역' 제거
    name = re.sub(r'\(.*?\)', '', str(name))
    name = re.sub(r'역$', '', name)
    return name.strip()

df_clean['역명'] = df_clean['역명'].apply(clean_station)

# 4. 결과 확인
display("정제된 Metro Inside 데이터", df_clean)


Saving metro_inside_data.csv to metro_inside_data (1).csv


'정제된 Metro Inside 데이터'

,역명,리프트,엘리베이터
0,동대문,0,0
1,종로3가,0,0
2,동대문,0,0
3,창신,1,0
4,창신,0,0
...,...,...,...
6558,고덕,0,0
6559,고덕,0,0
6560,고덕,0,0
6561,고덕,0,0


In [ ]:
from google.colab import files
import pandas as pd
import json

# 1) 파일 업로드
uploaded = files.upload()


Saving metro_els.json to metro_els.json
Saving metro_restroom.csv to metro_restroom.csv
Saving metro_ele_lift_count.json to metro_ele_lift_count.json


In [ ]:
# 2) Load each into a DataFrame and display columns
dataframes = {}

# Elevator & lift counts
ele_file = next(fn for fn in uploaded if 'lift' in fn or 'ele' in fn)
dataframes['ele_lift'] = pd.read_json(ele_file)
print(f"Columns in {ele_file}: {dataframes['ele_lift'].columns.tolist()}")
display(dataframes['ele_lift'].head())
ele_df = pd.read_json(ele_file)


Columns in metro_ele_lift_count.json: ['역명', '리프트', '엘리베이터']


,역명,리프트,엘리베이터
0,종로3가,6,3
1,동묘앞,5,2
2,수락산,4,2
3,일원,4,2
4,대청,4,2


In [ ]:
# Restroom info
rest_file = next(fn for fn in uploaded if 'restroom' in fn.lower())
dataframes['restroom'] = pd.read_csv(rest_file, encoding='cp949')
print(f"Columns in {rest_file}: {dataframes['restroom'].columns.tolist()}")
display(dataframes['restroom'].head())


Columns in metro_restroom.csv: ['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ', '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호', '상세위치 ', '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ', '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ', '남성용-어린이용소변기수 ', '여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ', '비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ', '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ', '기저귀교환대설치유무-여자장애인화장실 ', '리모델링 연도', '데이터기준일자']


,연번,구분,관리기관명,운영노선명,역명,소재지도로명주소,소재지지번주소,위도,경도,지상 또는 지하 구분,...,여성용-장애인용대변기수,여성용-어린이용대변기수,비상벨 설치유무,화장실입구cctv설치유무,기저귀교환대설치유무-남자화장실,기저귀교환대설치유무-남자장애인화장실,기저귀교환대설치유무-여자화장실,기저귀교환대설치유무-여자장애인화장실,리모델링 연도,데이터기준일자
0,1.0,공중화장실,서울교통공사,1호선,서울,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선),37.556878,126.972578,지하,...,1.0,2.0,Y,Y,Y,Y,Y,Y,2015,2025-03-13
1,2.0,공중화장실,서울교통공사,1호선,시청,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선),37.565682,126.976849,지하,...,1.0,2.0,Y,Y,Y,Y,Y,Y,2013,2025-03-13
2,3.0,공중화장실,서울교통공사,1호선,종각,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선),37.570175,126.983226,지하,...,1.0,1.0,Y,Y,Y,Y,Y,Y,2015,2025-03-13
3,4.0,공중화장실,서울교통공사,1호선,종각,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선),37.570175,126.983226,지하,...,1.0,NaN,Y,Y,Y,Y,Y,Y,2008이전,2025-03-13
4,5.0,공중화장실,서울교통공사,1호선,종로3가,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선),37.570420,126.992153,지하,...,1.0,1.0,Y,Y,Y,Y,Y,Y,2010,2025-03-13


In [ ]:
# Escalator data
els_file = next(fn for fn in uploaded if fn.lower().endswith('.json') and fn not in (ele_file,))
with open(els_file, 'r', encoding='utf-8') as f:
    esk_data = json.load(f).get('DATA', [])
dataframes['escalator'] = pd.DataFrame(esk_data)
print(f"Columns in {els_file}: {dataframes['escalator'].columns.tolist()}")
display(dataframes['escalator'].head())

Columns in metro_els.json: ['elvtr_no', 'eqpmnt', 'opr_drct', 'sbwy_rout_ln', 'instl_hgt', 'no', 'opr_len', 'plf_pbadms', 'opr_sec', 'sbwy_stns_nm']


,elvtr_no,eqpmnt,opr_drct,sbwy_rout_ln,instl_hgt,no,opr_len,plf_pbadms,opr_sec,sbwy_stns_nm
0,1800-448,ES,상행,1,4.6,1,11.6,None,B2-B1,서울역(1)
1,1806-600,ES,하행,1,4.6,2,11.6,None,B1-B2,서울역(1)
2,1806-599,ES,상행,1,4.6,3,11.6,None,B2-B1,서울역(1)
3,1810-816,ES,상행,1,6.8,4,16.0,None,B1-1F,서울역(1)
4,1810-817,ES,하행,1,6.8,5,16.0,None,1F-B1,서울역(1)


In [ ]:
# 3) Drop unwanted columns by specifying columns to keep
# Example: keep only station_name, elevator_count, lift_count for ele_lift
keep_ele = ['역명', '엘리베이터', '리프트']
dataframes['ele_lift'] = dataframes['ele_lift'][keep_ele]
dataframes['ele_lift'].columns = ['station_name', 'elevator_count', 'lift_count']

In [ ]:
# 2) 파일 이름 추출
rest_file = next(fn for fn in uploaded if 'restroom' in fn.lower())
els_file  = next(fn for fn in uploaded if fn.lower().endswith('.json'))

# 3) 화장실 데이터 로드 및 전처리
rest_df = pd.read_csv(rest_file, encoding='cp949')
# 필요한 컬럼만 추출 후 영어로 변경
rest_df = rest_df[['운영노선명 ', '역명', '게이트 내외 구분', '개방시간 ', '화장실입구cctv설치유무 ', '리모델링 연도']]
rest_df.columns = [
    'line',
    'station_name',
    'gate_access',
    'open_hours',
    'cctv_at_entrance',
    'remodel_year'
]

In [ ]:
# 4) 에스컬레이터 데이터 로드 및 전처리
with open(els_file, 'r', encoding='utf-8') as f:
    esk_data = json.load(f).get('DATA', [])
esk_df = pd.DataFrame(esk_data)
# 영어 컬럼명으로 변경
esk_df = esk_df.rename(columns={
    'opr_drct': 'direction',
    'sbwy_rout_ln': 'line',
    'instl_hgt': 'installation_height',
    'no': 'unit_no',
    'opr_len': 'escalator_length',
    'opr_sec': 'section',
    'sbwy_stns_nm': 'station_name'
})
# 필요한 컬럼만 추출
esk_df = esk_df[['direction', 'line', 'section', 'unit_no', 'installation_height', 'escalator_length', 'station_name']]


In [ ]:
esk_df

,direction,line,section,unit_no,installation_height,escalator_length,station_name
0,상행,1,B2-B1,1,4.6,11.6,서울역(1)
1,하행,1,B1-B2,2,4.6,11.6,서울역(1)
2,상행,1,B2-B1,3,4.6,11.6,서울역(1)
3,상행,1,B1-1F,4,6.8,16.0,서울역(1)
4,하행,1,1F-B1,5,6.8,16.0,서울역(1)
...,...,...,...,...,...,...,...
1866,상행,8,B2-B1,4,3.7,9.8,단대오거리
1867,상행,8,B1-1F,5,7.8,18.0,단대오거리
1868,하행,8,1F-B1,6,7.8,18.0,단대오거리
1869,하행,8,B1-B2,1,6.8,16.0,모란(8)


In [ ]:
# Display each processed DataFrame
print("=== Elevator & Lift Counts (Processed) ===")
display(ele_df.head(20))

print("\n=== Restroom Facilities (Processed) ===")
display(rest_df.head(20))

print("\n=== Escalator Details (Processed) ===")
display(esk_df.head(20))

=== Elevator & Lift Counts (Processed) ===


,역명,리프트,엘리베이터
0,종로3가,6,3
1,동묘앞,5,2
2,수락산,4,2
3,일원,4,2
4,대청,4,2
5,대치,4,1
6,남구로,3,0
7,월계,3,1
8,도곡,3,1
9,종로5가,2,0



=== Restroom Facilities (Processed) ===


,line,station_name,gate_access,open_hours,cctv_at_entrance,remodel_year
0,1호선,서울,외부,05:00~24:00,Y,2015
1,1호선,시청,외부,05:00~24:00,Y,2013
2,1호선,종각,외부,05:00~24:00,Y,2015
3,1호선,종각,외부,05:00~24:00,Y,2008이전
4,1호선,종로3가,외부,05:00~24:00,Y,2010
5,1호선,종로3가,내부,05:00~24:00,Y,2008이전
6,1호선,종로5가,외부,05:00~24:00,Y,2014
7,1호선,동대문,외부,05:00~24:00,Y,2016
8,1호선,동묘앞,외부,05:00~24:00,Y,2008이전
9,1호선,동묘앞,외부,05:00~24:00,Y,2008이전



=== Escalator Details (Processed) ===


,direction,line,section,unit_no,installation_height,escalator_length,station_name
0,상행,1,B2-B1,1,4.6,11.6,서울역(1)
1,하행,1,B1-B2,2,4.6,11.6,서울역(1)
2,상행,1,B2-B1,3,4.6,11.6,서울역(1)
3,상행,1,B1-1F,4,6.8,16.0,서울역(1)
4,하행,1,1F-B1,5,6.8,16.0,서울역(1)
5,하행,1,1F-B1,1,6.5,15.4,시청(1)
6,상행,1,B1-1F,2,6.5,15.4,시청(1)
7,상행,1,B1-1F,3,6.5,15.4,시청(1)
8,하행,1,1F-B1,1,7.0,16.4,종각
9,상행,1,B1-1F,2,7.0,16.4,종각


In [ ]:
from google.colab import files
import pandas as pd
import json

# 1) 세 파일 업로드
uploaded = files.upload()  # 창 뜨면 metro_ele_lift_count.json, metro_restroom.csv, metro_els.json 선택

# 2) 업로드된 파일명 파악
ele_file  = next(fn for fn in uploaded if 'lift' in fn.lower())
rest_file = next(fn for fn in uploaded if 'restroom' in fn.lower())
els_file  = next(fn for fn in uploaded if fn.lower().endswith('.json') and 'lift' not in fn.lower())


Saving metro_els.json to metro_els.json
Saving metro_ele_lift_count.json to metro_ele_lift_count.json
Saving metro_restroom.csv to metro_restroom.csv


In [ ]:
# 3) 엘리베이터·리프트 로드
ele_df = pd.read_json(ele_file)[['역명','리프트','엘리베이터']].copy()
ele_df.columns = ['station_name','lift_count','elevator_count']

# 4) 화장실 로드
rest_df = pd.read_csv(rest_file, encoding='cp949')[
    ['운영노선명 ','역명','게이트 내외 구분','개방시간 ','화장실입구cctv설치유무 ','리모델링 연도']
].copy()
rest_df.columns = ['line','station_name','gate_access','open_hours','cctv_at_entrance','remodel_year']

# 5) 에스컬레이터 로드
with open(els_file, 'r', encoding='utf-8') as f:
    esk_data = json.load(f).get('DATA', [])
esk_df = pd.DataFrame(esk_data)[[
    'sbwy_stns_nm','opr_drct','opr_sec','no','instl_hgt','opr_len'
]].rename(columns={
    'sbwy_stns_nm':'station_name',
    'opr_drct':'direction',
    'opr_sec':'section',
    'no':'unit_no',
    'instl_hgt':'installation_height',
    'opr_len':'escalator_length'
})


In [ ]:
# 6) 병합
merged = (ele_df
          .merge(rest_df, on='station_name', how='outer')
          .merge(esk_df, on='station_name', how='outer'))

# 7) 컬럼 순서 재배열
merged = merged[[
    'station_name','lift_count','elevator_count',
    'line','gate_access','open_hours','cctv_at_entrance','remodel_year',
    'direction','section','unit_no','installation_height','escalator_length'
]]


In [ ]:
# Save as JSON
merged.to_json('combined_facilities.json', orient='records', force_ascii=False, indent=2)

# Download JSON
files.download('combined_facilities.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 정리된 버전의 코드

In [ ]:
import pandas as pd
import io
import json
import re
from functools import reduce

# ---------------------------------------------
# 1. 유틸리티 함수 정의
# ---------------------------------------------

def load_csv(path, encodings=('utf-8', 'cp949', 'euc-kr', 'latin1')):
    """
    다양한 인코딩을 시도하여 CSV 파일을 읽어옵니다.
    실패 시 on_bad_lines='warn' 옵션으로 경고만 출력합니다.
    """
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, on_bad_lines='warn')
        except Exception:
            continue
    raise UnicodeDecodeError(f"Unable to read CSV: {path}")


def load_json(path, key='DATA'):
    """
    JSON 파일에서 지정한 키(DATA 등)가 있는 경우 해당 데이터를 DataFrame으로 변환
    키가 없으면 전체 객체를 사용합니다.
    """
    with open(path, 'r', encoding='utf-8') as f:
        obj = json.load(f)
    records = obj.get(key, obj)
    return pd.json_normalize(records)

# ---------------------------------------------
# 2. 원본 데이터 로드
# ---------------------------------------------

df_inside    = load_csv('metro_inside_data.csv')    # 내부 편의시설
df_lift      = load_json('metro_lift.json')         # 리프트 정보
df_elev      = load_json('metro_elevator.json')     # 엘리베이터 정보
df_acc       = load_csv('metro_accident.csv')       # 사고 발생 정보
df_late      = load_csv('metro_late.csv')           # 지연 정보
# 추가 2개 파일
df_restroom  = load_csv('metro_restroom.csv')       # 화장실 정보
df_escalator = load_json('metro_els.json')          # 에스컬레이터 정보
# df_depth    = load_csv('metro_depth.csv')        # 심도 정보 (4.6에서 처리)

# ---------------------------------------------
# 3. 공통: 역명 정제 함수
# ---------------------------------------------

def clean_station_name(name):
    """
    1) 괄호 및 내부 텍스트 제거
    2) '역' 접미사 제거
    3) 양끝 공백 제거
    """
    s = str(name)
    s = re.sub(r"\(.*?\)", '', s)
    s = re.sub(r"역$", '', s)
    return s.strip()


In [ ]:
# ---------------------------------------------
# 4. 개별 데이터 세부 전처리
# ---------------------------------------------

# 4.1 내부 편의시설 데이터
#  - 지하철역명 → 역명, 리프트·엘리베이터 결측치 채우기, 그룹 집계

df_in = (
    df_inside
    .loc[:, ['지하철역명', '리프트', '엘리베이터']]
    .rename(columns={'지하철역명': '역명'})
)
df_in['역명'] = df_in['역명'].apply(clean_station_name)
df_in[['리프트', '엘리베이터']] = df_in[['리프트', '엘리베이터']].fillna(0)
station_inside = df_in.groupby('역명', as_index=False).sum()

# 4.2 리프트 정보
#  - sbwy_stn_nm → 역명, node_type_cd → lift_count, 정수형 변환 및 그룹 집계

df_l = df_lift.rename(columns={'sbwy_stn_nm': '역명', 'node_type_cd': 'lift_count'})
df_l['역명'] = df_l['역명'].apply(clean_station_name)
df_l['lift_count'] = pd.to_numeric(df_l['lift_count'], errors='coerce').fillna(0).astype(int)
station_lift = df_l.groupby('역명', as_index=False)['lift_count'].sum()

# 4.3 엘리베이터 정보
#  - sbwy_stn_nm → 역명, node_type_cd → elevator_count, 정수형 변환 및 그룹 집계

df_e = df_elev.rename(columns={'sbwy_stn_nm': '역명', 'node_type_cd': 'elevator_count'})
df_e['역명'] = df_e['역명'].apply(clean_station_name)
df_e['elevator_count'] = pd.to_numeric(df_e['elevator_count'], errors='coerce').fillna(0).astype(int)
station_elev = df_e.groupby('역명', as_index=False)['elevator_count'].sum()

# 4.4 사고 정보
#  - 사고유형·발생역 필터링, 역명 정제 후 사고 건수 집계

df_a = df_acc[['사고유형', '발생역']].dropna()
df_a['역명'] = df_a['발생역'].apply(clean_station_name)
station_acc = df_a.groupby('역명').size().reset_index(name='accident_count')

# 4.5 지연 정보
#  - 번호 컬럼 제거, 최대지연시간(분)에서 숫자 추출, 평균 지연시간 계산

df_late2 = df_late.drop(columns=['번호'], errors='ignore')
df_late2['delay_min'] = (
    df_late2['최대지연시간(분)']
    .astype(str)
    .str.extract(r"(\d+)")[0]
    .astype(int)
)
station_delay = (
    df_late2
    .groupby(df_late2['노선'], as_index=False)['delay_min']
    .mean()
    .rename(columns={'노선': '역명', 'delay_min': 'avg_delay'})
)
station_delay['역명'] = station_delay['역명'].apply(clean_station_name)

# 4.6 심도 정보
#  - station_cd, depth_m 컬럼 추출
#  - depth_m 숫자로 변환, station_cd→역명 매핑, 평균 심도 계산

df_depth = load_csv('metro_depth.csv')
df_depth2 = df_depth[['station_cd', 'depth_m']].copy()
df_depth2['depth_m'] = (
    df_depth2['depth_m'].astype(str)
    .str.extract(r"(\d+\.?\d*)")[0]
    .astype(float)
)
mapping = load_json('metro_inside_info.json')[['station_cd', 'station_name']]
mapping['station_cd'] = mapping['station_cd'].astype(str)
mapping['역명'] = mapping['station_name'].apply(clean_station_name)
df_depth3 = (
    df_depth2.assign(station_cd=df_depth2['station_cd'].astype(str))
    .merge(mapping[['station_cd','역명']], on='station_cd', how='left')
)
station_depth = df_depth3.groupby('역명', as_index=False)['depth_m'].mean().rename(columns={'depth_m':'avg_depth_m'})

# 4.7 화장실 정보
#  - 운영노선명·역명·게이트·개방시간·CCTV·리모델링 연도 선택
#  - 컬럼명 영문 변환, 역명 정제, 그룹별 최신 모델 연도, CCTV 설치 여부 정리

df_r = df_restroom.copy()
df_r = df_r[[ '운영노선명 ', '역명', '게이트 내외 구분', '개방시간 ', '화장실입구cctv설치유무 ', '리모델링 연도']]
df_r.columns = ['line','station_name','gate_access','open_hours','cctv_at_entrance','remodel_year']
df_r['station_name'] = df_r['station_name'].apply(clean_station_name)
# 리모델링 연도 최대값, CCTV 설치 여부(Y/N) 등 요약
station_restroom = (
    df_r.groupby('station_name', as_index=False).agg({
        'line':'first',
        'gate_access':'first',
        'open_hours':'first',
        'cctv_at_entrance':lambda x: 'Y' if any(x=='Y') else 'N',
        'remodel_year':'max'
    }).rename(columns={'station_name':'역명'})
)

# 4.8 에스컬레이터 정보
#  - JSON DATA에서 필요한 컬럼 추출 및 영문명 변경
#  - 역명 정제, 설치 높이·길이 평균, 대수 집계

df_esk = df_escalator[['sbwy_stns_nm','opr_drct','opr_sec','no','instl_hgt','opr_len']].copy()
df_esk = df_esk.rename(columns={
    'sbwy_stns_nm':'station_name',
    'opr_drct':'direction',
    'opr_sec':'section',
    'no':'unit_no',
    'instl_hgt':'installation_height',
    'opr_len':'escalator_length'
})
df_esk['station_name'] = df_esk['station_name'].apply(clean_station_name)
# 역명별 평균 설치 높이, 총 에스컬레이터 대수, 평균 길이
station_escalator = (
    df_esk.groupby('station_name', as_index=False).agg({
        'installation_height':'mean',
        'escalator_length':'mean',
        'unit_no':'count'
    }).rename(columns={
        'station_name':'역명',
        'unit_no':'escalator_count',
        'installation_height':'avg_install_hgt',
        'escalator_length':'avg_esk_length'
    })
)


In [ ]:
# ---------------------------------------------
# 5. 데이터 통합 (역명 기준 outer 조인)
# ---------------------------------------------

df_list = [
    station_inside, station_lift, station_elev,
    station_acc, station_delay, station_depth,
    station_restroom, station_escalator
]
df_merged = reduce(lambda left,right: pd.merge(left,right,on='역명',how='outer'), df_list)

# NaN을 기본값으로 대체
defaults = {
    '리프트':0, '엘리베이터':0,
    'lift_count':0, 'elevator_count':0,
    'accident_count':0, 'avg_delay':0,
    'avg_depth_m':0,
    'cctv_at_entrance':'N', 'remodel_year':0,
    'escalator_count':0, 'avg_install_hgt':0, 'avg_esk_length':0
}
df_merged = df_merged.fillna(defaults)

# ---------------------------------------------
# 6. 결과 저장 및 확인 (파일명 변경)
# ---------------------------------------------

df_merged.to_json('metro_inside.json', orient='records', force_ascii=False, indent=2)
print("metro_inside.json' 저장 완료")


## **4-1) 혼잡도_ 지하철 혼잡도 정보**
지하철 혼잡도 데이터를 담고 있는 서울교통공사_지하철혼잡도정보_20241231.csv와 서울우교통공사_지하철혼잡도정보_20240630.csv 파일을 전처리하는 작업에서는
* csv 데이터프레임을 json 형태로 변환하여 저장

In [ ]:
import pandas as pd
from google.colab import files

# 1. 인터랙티브하게 파일 업로드
uploaded = files.upload()  # 업로드 창이 뜹니다.


Saving 서울교통공사_지하철혼잡도정보_20241231.csv to 서울교통공사_지하철혼잡도정보_20241231.csv
Saving 서울교통공사_지하철혼잡도정보_20240630.csv to 서울교통공사_지하철혼잡도정보_20240630.csv


In [ ]:
# 2. 업로드된 첫 번째 파일을 DataFrame으로 읽기
file_name = next(iter(uploaded.keys()))
# 인코딩이 utf-8이 아니면 cp949으로 변경해보세요.
df = pd.read_csv(file_name, encoding='cp949')


In [ ]:
df

,연번,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,1,평일,1,150,서울역,상선,8.6,20.9,20.4,33.1,...,19.3,18.6,19.1,15.4,16.7,18.1,13.0,12.4,6.5,1.0
1,2,평일,1,151,시청,상선,8.6,15.6,17.0,26.8,...,23.2,23.0,23.8,18.6,20.9,20.7,17.7,14.0,7.0,1.1
2,3,평일,1,152,종각,상선,8.4,12.5,10.8,18.1,...,30.1,29.7,32.6,24.7,27.4,26.7,24.4,16.2,7.8,1.1
3,4,평일,1,153,종로3가,상선,7.2,13.8,10.9,18.7,...,30.6,32.0,38.7,27.4,30.6,30.4,27.3,18.3,8.0,1.1
4,5,평일,1,154,종로5가,상선,6.6,12.4,8.8,14.0,...,32.4,33.5,39.2,33.5,31.8,28.1,27.5,17.8,12.8,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,1658,일요일,8,2826,수진,하선,9.6,4.0,4.6,4.0,...,2.7,1.9,2.2,1.8,1.6,1.3,0.8,0.6,0.1,0.0
1658,1659,일요일,8,2827,모란,상선,1.4,1.0,2.2,1.5,...,5.9,6.2,6.2,5.9,4.5,4.5,3.3,4.2,0.0,0.0
1659,1660,일요일,8,2827,모란,하선,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1660,1661,일요일,8,2828,남위례,상선,11.9,8.8,8.6,10.6,...,8.9,7.8,7.3,7.1,6.5,5.3,3.4,1.8,0.0,0.0


In [ ]:
# 3. DataFrame을 JSON으로 변환하여 로컬에 저장
json_filename = file_name.rsplit('.', 1)[0] + '.json'
df.to_json(json_filename, orient='records', force_ascii=False, indent=2)

# 4. 생성된 JSON 파일을 다운로드
files.download(json_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files

# 1. 인터랙티브하게 파일 업로드
uploaded = files.upload()  # 업로드 창이 뜹니다.

# 2. 업로드된 첫 번째 파일을 DataFrame으로 읽기
file_name = next(iter(uploaded.keys()))
# 인코딩이 utf-8이 아니면 cp949으로 변경해보세요.
df = pd.read_csv(file_name, encoding='cp949')

# 3. DataFrame을 JSON으로 변환하여 로컬에 저장
json_filename = file_name.rsplit('.', 1)[0] + '.json'
df.to_json(json_filename, orient='records', force_ascii=False, indent=2)

# 4. 생성된 JSON 파일을 다운로드
files.download(json_filename)


Saving metro_transfer_time.csv to metro_transfer_time (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
import pandas as pd

# 1) Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2) 마운트된 드라이브 폴더 안 파일 경로 지정
path = '/Users/kodaewoong/Desktop/study_file/bus/all/metro/편의성/combined_facilities.json'

In [ ]:
# 3) 판다스로 읽기
df = pd.read_csv(path, encoding='cp949')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kodaewoong/Desktop/study_file/bus/all/metro/편의성/combined_facilities.json'

In [ ]:
import pandas as pd
from google.colab import files

# 1. 인터랙티브하게 파일 업로드
uploaded = files.upload()  # 업로드 창이 뜹니다.

Saving 서울교통공사_지하철혼잡도정보_20250331.xlsx to 서울교통공사_지하철혼잡도정보_20250331.xlsx


역 정보 + a

## **4-2) 혼잡도_ 승하차인원 정보**
지하철 혼잡도에 기여하는 승하차인원 정보 데이터를 담고 있는 ['metro_people_1.csv', 'metro_people_2.csv', 'metro_people_3.csv', 'metro_people_4.csv', 'metro_people_5.csv', 'metro_people_6.csv', 'metro_people_7.csv', 'metro_people_8.csv', 'metro_people_9.csv', 'metro_people_10.csv', 'metro_people_11.csv', 'metro_people_12.csv'] 파일을 전처리하는 작업에서는
* 12개 파일 각각의 칼럼 ['사용년월', '노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '교통수단타입코드', '교통수단타입명', '등록일자']
중에서 [
    '노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명',
    '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수',
    '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수',
    '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수',
    '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수',
    '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수',
    '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수',
    '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수',
    '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수',
    '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수',
    '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수',
    '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수',
    '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수',
    '교통수단타입코드'
]만 추출
* (1월 ~ 12월, 파일명) 형태의 리스트 구성
* 해당 월 숫자 값을 'month'라는 새 컬럼으로 추가하여 정의한 컬럼(55개)만 선택하여 데이터프레임 구성
* 월별 처리된 데이터프레임들을 concat()으로 병합하여 총 12개월치 승하차 인원 데이터를 하나의 데이터프레임으로 생성

지하철 혼잡도에 기여하는 월별 승하차인원 정보 데이터를 담고 있는 CARD_SUBWAY_MONTH_2022.csv 파일과 서울교통공사_지하철혼잡도정보_20250331.xlsx 파일 또한 반영하였음



In [ ]:
# 업로드가 완료되면 어떤 파일이 올라왔는지 확인합니다.
print("▶ 업로드된 파일 목록:", list(uploaded.keys()))

# 2) pandas로 샘플 파일 하나를 읽어서 구조를 살펴봅니다.
import pandas as pd

# 업로드된 파일 중 첫 번째 파일명을 가져옵니다.
sample_file = list(uploaded.keys())[0]
print("▶ 읽어들일 파일:", sample_file)

# utf-8로 시도, 실패 시 cp949로 재시도
try:
    df_sample = pd.read_csv(sample_file)
    print("✅ utf-8 인코딩으로 읽기 성공")
except UnicodeDecodeError:
    print("⚠️ utf-8으로 읽기 실패, cp949로 재시도합니다...")
    df_sample = pd.read_csv(sample_file, encoding='cp949')
    print("✅ cp949 인코딩으로 읽기 성공")

# 컬럼 목록 출력
print("\n▶ 샘플 파일의 컬럼들:")
print(df_sample.columns.tolist())

# 데이터 미리보기
print("\n▶ 샘플 데이터 첫 5행:")
print(df_sample.head())

▶ 업로드된 파일 목록: ['metro_people_1.csv', 'metro_people_2.csv', 'metro_people_3.csv', 'metro_people_4.csv', 'metro_people_5.csv', 'metro_people_6.csv', 'metro_people_7.csv', 'metro_people_8.csv', 'metro_people_9.csv', 'metro_people_10.csv', 'metro_people_11.csv', 'metro_people_12.csv']
▶ 읽어들일 파일: metro_people_1.csv
⚠️ utf-8으로 읽기 실패, cp949로 재시도합니다...
✅ cp949 인코딩으로 읽기 성공

▶ 샘플 파일의 컬럼들:
['사용년월', '노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총

<ipython-input-3-2da10ca62819>:17: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sample = pd.read_csv(sample_file, encoding='cp949')


In [ ]:
# 3) 1~8월 데이터 불러오기 → 컬럼 추출 → 합치기 → JSON 저장
import pandas as pd
import json

# 1) 남길 컬럼 리스트
keep_cols = [
    '노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명',
    '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수',
    '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수',
    '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수',
    '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수',
    '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수',
    '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수',
    '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수',
    '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수',
    '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수',
    '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수',
    '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수',
    '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수',
    '교통수단타입코드'
]

In [ ]:
# 2) (월, 파일명) 리스트 생성
file_list = [(i, f'metro_people_{i}.csv') for i in range(1, 13)]

dfs = []
for month, fname in file_list:
    print(f"\n▶ {month}월 파일 읽는 중: {fname}")
    try:
        df = pd.read_csv(fname)
        print("   - utf-8 로 읽기 성공")
    except UnicodeDecodeError:
        print("   - utf-8 실패 → cp949 로 재시도")
        df = pd.read_csv(fname, encoding='cp949')
        print("   - cp949 로 읽기 성공")

    # 월 정보 추가
    df['month'] = month
    print(f"   - month 컬럼 추가 (값={month})")

    # 필요 컬럼만 선택
    df_sel = df[keep_cols]
    print(f"   - 선택된 컬럼: {df_sel.columns.tolist()}")
    print("   - 샘플 2행:")
    print(df_sel.head(2))

    dfs.append(df_sel)

# 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)
print(f"\n▶ 전체 합친 후 DataFrame 형태: {df_all.shape}")
print(df_all.head(5))



▶ 1월 파일 읽는 중: metro_people_1.csv
   - utf-8 실패 → cp949 로 재시도


<ipython-input-7-85409bf3fd36>:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, encoding='cp949')


   - cp949 로 읽기 성공
   - month 컬럼 추가 (값=1)
   - 선택된 컬럼: ['노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '교통수단타입코드']
   - 샘플 2행:
  노선번호                    노선명  표준버스정류장ID 버스정류장ARS번호              역명  \
0  470       470번(상암차고지~안골마을)  100000001       1001  종로2가사거리(00066)   
1  N37  N37번(진관공영차고지~송파공영차고지)  100000001       1001  종로2가사거리(00089)   

   00시승차총승객수  00시하차총승객수  1시승차총승객수  1시하차총승객수  2시승차총승객

<ipython-input-7-85409bf3fd36>:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, encoding='cp949')


   - cp949 로 읽기 성공
   - month 컬럼 추가 (값=2)
   - 선택된 컬럼: ['노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '교통수단타입코드']
   - 샘플 2행:
  노선번호                    노선명  표준버스정류장ID 버스정류장ARS번호              역명  \
0  741      741번(진관차고지~헌인릉입구)  100000001       1001  종로2가사거리(00075)   
1  N37  N37번(송파공영차고지~진관공영차고지)  100000001       1001  종로2가사거리(00032)   

   00시승차총승객수  00시하차총승객수  1시승차총승객수  1시하차총승객수  2시승차총승객

...

<ipython-input-7-85409bf3fd36>:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, encoding='cp949')


   - cp949 로 읽기 성공
   - month 컬럼 추가 (값=7)
   - 선택된 컬럼: ['노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '교통수단타입코드']
   - 샘플 2행:
  노선번호                노선명  표준버스정류장ID 버스정류장ARS번호              역명  00시승차총승객수  \
0  470   470번(상암차고지~안골마을)  100000001       1001  종로2가사거리(00066)         59   
1  741  741번(진관차고지~헌인릉입구)  100000001       1001  종로2가사거리(00075)        147   

   00시하차총승객수  1시승차총승객수  1시하차총승객

<ipython-input-7-85409bf3fd36>:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, encoding='cp949')


   - cp949 로 읽기 성공
   - month 컬럼 추가 (값=10)
   - 선택된 컬럼: ['노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수', '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수', '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수', '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수', '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수', '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수', '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수', '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수', '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수', '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '교통수단타입코드']
   - 샘플 2행:
  노선번호                    노선명  표준버스정류장ID 버스정류장ARS번호              역명  \
0  470       470번(상암차고지~안골마을)  100000001       1001  종로2가사거리(00066)   
1  N37  N37번(송파공영차고지~진관공영차고지)  100000001       1001  종로2가사거리(00032)   

   00시승차총승객수  00시하차총승객수  1시승차총승객수  1시하차총승객수  2시승차총승

<ipython-input-7-85409bf3fd36>:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fname, encoding='cp949')


In [ ]:
# JSON 변환 및 저장
json_records = df_all.to_json(orient='records', force_ascii=False)
with open('metro_1-8월_승하차데이터_월별.json', 'w', encoding='utf-8') as f:
    f.write(json_records)
print("\n▶ JSON 파일로 저장 완료: metro_1-8월_승하차데이터_월별.json")
print("\n▶ JSON 샘플 (처음 300자):")
print(json_records[:300])


▶ JSON 파일로 저장 완료: metro_1-8월_승하차데이터_월별.json

▶ JSON 샘플 (처음 300자):
[{"노선번호":"470","노선명":"470번(상암차고지~안골마을)","표준버스정류장ID":100000001,"버스정류장ARS번호":1001,"역명":"종로2가사거리(00066)","00시승차총승객수":63,"00시하차총승객수":99,"1시승차총승객수":0,"1시하차총승객수":0,"2시승차총승객수":0,"2시하차총승객수":0,"3시승차총승객수":0,"3시하차총승객수":0,"4시승차총승객수":0,"4시하차총승객수":0,"5시승차총승객수":0,"5시하차총승객수":0,"6시승차총승객수":213,"6시하차총승객수":80,"7시승차총승객수


In [ ]:
from google.colab import files

# 1. 파일 업로드 실행
uploaded = files.upload()
# 업로드가 완료되면 어떤 파일이 올라왔는지 확인합니다.
print("▶ 업로드된 파일 목록:", list(uploaded.keys()))

# 2) pandas로 샘플 파일 하나를 읽어서 구조를 살펴봅니다.
import pandas as pd

# 업로드된 파일 중 첫 번째 파일명을 가져옵니다.
sample_file = list(uploaded.keys())[0]
print("▶ 읽어들일 파일:", sample_file)

# utf-8로 시도, 실패 시 cp949로 재시도
try:
    df_sample = pd.read_csv(sample_file)
    print("✅ utf-8 인코딩으로 읽기 성공")
except UnicodeDecodeError:
    print("⚠️ utf-8으로 읽기 실패, cp949로 재시도합니다...")
    df_sample = pd.read_csv(sample_file, encoding='cp949')
    print("✅ cp949 인코딩으로 읽기 성공")

# 컬럼 목록 출력
print("\n▶ 샘플 파일의 컬럼들:")
print(df_sample.columns.tolist())

# 데이터 미리보기
print("\n▶ 샘플 데이터 첫 5행:")
print(df_sample.head())

Saving CARD_SUBWAY_MONTH_2022.csv to CARD_SUBWAY_MONTH_2022.csv
▶ 업로드된 파일 목록: ['CARD_SUBWAY_MONTH_2022.csv']
▶ 읽어들일 파일: CARD_SUBWAY_MONTH_2022.csv
⚠️ utf-8으로 읽기 실패, cp949로 재시도합니다...
✅ cp949 인코딩으로 읽기 성공

▶ 샘플 파일의 컬럼들:
['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자']

▶ 샘플 데이터 첫 5행:
         사용일자  노선명    역명  승차총승객수    하차총승객수  등록일자
20220101  3호선   수서  7370    7076  20220104   NaN
20220101  3호선  학여울   461     473  20220104   NaN
20220101  3호선   대청  3224    2903  20220104   NaN
20220101  3호선   일원  3321    2803  20220104   NaN
20220101  경원선   창동     1       0  20220104   NaN


In [ ]:
print(df_sample.tail())

         사용일자      노선명     역명  승차총승객수    하차총승객수  등록일자
20221231  4호선      신용산  11025   12099  20230103   NaN
20221231  경부선       석수   5643    5147  20230103   NaN
20221231  경부선     금천구청   7519    7426  20230103   NaN
20221231  경부선  가산디지털단지   5579    7299  20230103   NaN
20221231  경부선       구로  11318   12080  20230103   NaN


In [ ]:
from google.colab import files

# 1. 파일 업로드 실행
uploaded = files.upload()
# 업로드가 완료되면 어떤 파일이 올라왔는지 확인합니다.
print("▶ 업로드된 파일 목록:", list(uploaded.keys()))

# 2) pandas로 샘플 파일 하나를 읽어서 구조를 살펴봅니다.
import pandas as pd

# 업로드된 파일 중 첫 번째 파일명을 가져옵니다.
sample_file = list(uploaded.keys())[0]
print("▶ 읽어들일 파일:", sample_file)

# utf-8로 시도, 실패 시 cp949로 재시도
try:
    df_sample = pd.read_csv(sample_file)
    print("✅ utf-8 인코딩으로 읽기 성공")
except UnicodeDecodeError:
    print("⚠️ utf-8으로 읽기 실패, cp949로 재시도합니다...")
    df_sample = pd.read_csv(sample_file, encoding='cp949')
    print("✅ cp949 인코딩으로 읽기 성공")

# 컬럼 목록 출력
print("\n▶ 샘플 파일의 컬럼들:")
print(df_sample.columns.tolist())

# 데이터 미리보기
print("\n▶ 샘플 데이터 첫 5행:")
print(df_sample.head())
print(df_sample.tail())

Saving please.csv to please.csv
▶ 업로드된 파일 목록: ['please.csv']
▶ 읽어들일 파일: please.csv
⚠️ utf-8으로 읽기 실패, cp949로 재시도합니다...
✅ cp949 인코딩으로 읽기 성공

▶ 샘플 파일의 컬럼들:
['연번', '수송일자', '호선', '역번호', '역명', '승하차구분', '06시이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시이후']

▶ 샘플 데이터 첫 5행:
   연번        수송일자   호선  역번호   역명 승하차구분  06시이전  06-07시간대  07-08시간대  08-09시간대  \
0   1  2023-01-01  1호선  150  서울역    승차    215       145       231       594   
1   2  2023-01-01  1호선  150  서울역    하차    154       636       595       939   
2   3  2023-01-01  1호선  151   시청    승차     48        73       106       194   
3   4  2023-01-01  1호선  151   시청    하차     64       247       293       463   
4   5  2023-01-01  1호선  152   종각    승차    407       235       158       201   

   ...  15-16시간대  16-17시간대  17-18시간대  18-19시간대  19-20시간대  20-21시간대  21-22시간대  \

In [ ]:
# 1. Read the Excel file
file_path = '서울교통공사_지하철혼잡도정보_20250331.xlsx'
df = pd.read_excel(file_path)

In [ ]:
output_path = '서울교통공사_지하철혼잡도정보_20250331.json'
df.to_json(output_path, orient='records', force_ascii=False, indent=2)

In [ ]:
# 3. Show the first few rows as a preview
df.head()

,연번,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,1,평일,1,158,청량리,상선,7.2,6.9,4.5,8.3,...,24.8,26.1,28.2,24.5,23.0,22.2,21.7,14.9,8.5,0.0
1,2,평일,1,157,제기동,상선,7.6,8.7,6.5,8.7,...,30.0,26.0,34.8,27.5,25.7,25.4,24.2,16.8,11.6,0.0
2,3,평일,1,156,신설동,상선,6.7,11.2,7.2,9.6,...,30.7,26.8,36.3,28.6,26.6,26.1,25.2,16.1,12.6,0.0
3,4,평일,1,159,동묘앞,상선,6.3,11.8,7.4,12.2,...,32.1,30.1,41.8,29.9,29.0,23.5,27.7,13.5,14.3,0.0
4,5,평일,1,155,동대문,상선,7.4,11.2,8.3,14.0,...,35.6,33.5,40.5,34.7,32.6,26.1,31.3,18.0,13.6,3.5


### 정리된 버전의 코드

In [ ]:
# ---------------------------------------------
# 1. 라이브러리 임포트
# ---------------------------------------------
import io
import re
import pandas as pd
from google.colab import files
from IPython.display import display

# ---------------------------------------------
# 2. 1~12월 지하철 혼잡도 CSV 파일 업로드
# ---------------------------------------------
print("▶ metro_people_1.csv ~ metro_people_12.csv 파일을 모두 선택하여 업로드하세요.")
uploaded = files.upload()  # 업로드 창에서 12개 파일 선택

# ---------------------------------------------
# 3. 파일별로 읽어 '월' 컬럼 추가 후 통합
# ---------------------------------------------
dfs = []
for fname, content in uploaded.items():
    # 파일명에서 '_<월>.csv' 형태로 월 정보 추출
    m = re.search(r'_(\d+)\.csv$', fname)
    if not m:
        print(f"⚠️ 파일명에 월 정보가 없습니다: {fname}, 건너뜁니다.")
        continue
    month = int(m.group(1))

    # CSV 읽기 (한글 인코딩 고려)
    try:
        df = pd.read_csv(io.BytesIO(content), encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(io.BytesIO(content), encoding='cp949')

    # 월 정보 컬럼 추가
    df['월'] = month
    dfs.append(df)

# 모든 월 데이터 하나로 합치기
combined = pd.concat(dfs, ignore_index=True)
print(f"▶ 통합 완료: {combined.shape[0]}개 레코드, 컬럼 {combined.columns.tolist()}")
display(combined.head())

# ---------------------------------------------
# 4. 컬럼명 정리 및 역명 추출
# ---------------------------------------------
# 원본 컬럼 예시: ['연번','요일구분','호선','역번호','출발역','상하구분','5시30분',...]
#  - '출발역' → 'station_name'
#  - '호선' → 'line'
#  - '상하구분' → 'direction'
combined = combined.rename(columns={
    '출발역': 'station_name',
    '호선': 'line',
    '상하구분': 'direction'
})

# 시간대 컬럼만 골라내기
time_cols = [col for col in combined.columns if re.match(r'\d+시\d{2}분', col)]
print("▶ 시간대 컬럼:", time_cols[:5], "… 총", len(time_cols), "개")

# ---------------------------------------------
# 5. long 형식으로 변환(melt) 후 연간 평균 계산
# ---------------------------------------------
# melt: 한 레코드당 하나의 시간대·혼잡도 값 갖도록 변환
melted = combined.melt(
    id_vars=['월', 'station_name', 'line', 'direction'],
    value_vars=time_cols,
    var_name='time',
    value_name='congestion'
)

# 월별 중복 제거(필요시), NaN 제거
melted = melted.dropna(subset=['congestion'])

# station_name, line, direction, time별 연평균 혼잡도 계산
yearly = (
    melted
    .groupby(['station_name', 'line', 'direction', 'time'], as_index=False)
    ['congestion']
    .mean()
    .rename(columns={'congestion': 'avg_congestion'})
)

print(f"▶ 연간 평균 계산 완료: {yearly.shape[0]}개 레코드")
display(yearly.head())

# ---------------------------------------------
# 6. wide 형식으로 재구성(pivot)—옵션
# ---------------------------------------------
yearly_wide = yearly.pivot_table(
    index=['station_name', 'line', 'direction'],
    columns='time',
    values='avg_congestion'
).reset_index()

# 컬럼 이름 정리
yearly_wide.columns.name = None
print(f"▶ wide 형식 변환 완료: {yearly_wide.shape[0]}개 역, 컬럼 {yearly_wide.columns.tolist()[:6]}…")
display(yearly_wide.head())

# ---------------------------------------------
# 7. CSV 및 JSON 저장·다운로드
# ---------------------------------------------
csv_out = 'metro_congestion_yearly.csv'
json_out = 'metro_congestion_yearly.json'

yearly_wide.to_csv(csv_out, index=False, encoding='utf-8-sig')
print(f"✅ '{csv_out}' 저장 완료")

yearly_wide.to_json(json_out, orient='records', force_ascii=False, indent=2)
print(f"✅ '{json_out}' 저장 완료")

files.download(csv_out)
files.download(json_out)


## **5-1) 쾌적성_계절 및 온도 데이터**
* json 형태로 되어있었던 각 weather_spring.json, weather_summer.json, weather_fall.json, weather_winter.json파일을 import
* 각 파일의 봄,여름,가을,겨울 지역 및 온도 데이터의 칼럼 명을 대문자로 변환
* 3•4•5월을 봄, 6•7•8월을 여름, 9•10•11월을 가을, 그리고 나머지 달을 겨울로 지정하여 월에 계절 지정 및 계절 칼럼 생성
* 편의성을 위하여 공식 날씨 관측소 코드인 'aws_code' 및 'AWS_CDE'를 'observatory_id'로 통일하고, 구 단위를 나타내는 'objectid'와 'OBJECTID'는 'region_id'로 통일하여 칼럼명 변경
* 위치 관련 칼럼인 'X', 'LNG'를 'lng'로 통일하고, 'Y', 'LAT'칼럼은 'lat'으로 통합하여 칼럼명 변경
* ['observatory_id', 'region_id', 'nam', 'address', 'lng', 'lat']로 칼럼 순서 재정렬
* 함수화 방식
```
#기온 점수: 현재 계절의 역 주변 평균 기온을 "쾌적(4점) ~ 혹한/혹서(1점)"의 4단계로 점수화
def score_temp_cat(x):
 if 10 <= x <= 24: return 4
 elif (0 <= x < 10) or (24 < x <= 28): return 3
 elif (-10 <= x < 0) or (28 < x <= 34): return 2
 else: return 1
 ```
* pd.concat()을 통해 계절별 DataFrame을 하나로 병합


In [ ]:
import pandas as pd
import json

# JSON 파일 로드
with open('weather_spring.json', 'r', encoding='utf-8') as f:
    weather = json.load(f)

# 'DATA' 부분을 DataFrame으로 변환
df = pd.DataFrame(weather['DATA'])

# 컬럼명을 대문자로 변경 (필요시)
df.rename(columns={
    'aws_cde': 'AWS_CDE',
    'nam': 'NAM',
    'address': 'ADDRESS',
    'lng': 'LNG',
    'objectid': 'OBJECTID',
    'lat': 'LAT',
    'spr_temp': 'SPR_TEMP'
}, inplace=True)

# 엑셀 파일로 저장
output_path = 'weather_spring.xlsx'
df.to_excel(output_path, index=False)

print(f"Excel 파일이 생성되었습니다: {output_path}")

Excel 파일이 생성되었습니다: weather_spring.xlsx


In [ ]:
import json
import pandas as pd

# 파일 경로 지정
file_paths = {
    'spring': '/content/weather_spring.json',
    'summer': '/content/weather_summer.json',
    'fall': '/content/weather_fall.json',
    'winter': '/content/weather_winter.json'
}

def month_to_season(m):
    if m in [3, 4, 5]:
        return 'spring'
    elif m in [6, 7, 8]:
        return 'summer'
    elif m in [9, 10, 11]:
        return 'fall'
    else:
        return 'winter'

dfs = []
for season, path in file_paths.items():
    with open(path, encoding='utf-8') as f:
        data = json.load(f)
        # 데이터 키가 DATA임
        df = pd.DataFrame(data['DATA'])
        df['season'] = season
        dfs.append(df)

# 모든 계절 데이터 통합
df_all = pd.concat(dfs, ignore_index=True)

# 파일로 저장 (옵션)
df_all.to_csv('/content/weather_seasons.csv', index=False, encoding='utf-8-sig')

# 결과 일부 출력
print(df_all.head())


   aws_cde  nam                       address          lng  objectid  \
0    509.0   관악   서울특별시 관악구 신림동 산56-1 (서울대학교)  126.9502188         1   
1    417.0   금천   서울특별시 금천구 독산동 1034 (독산초등학교)  126.9141209         2   
2    401.0   서초  서울특별시 서초구 서초동 1650 (서울교육대학교)  127.0179133         3   
3    423.0   구로  서울특별시 구로구 궁동 213-42 (수궁동사무소)  126.8312293         4   
4    410.0  기상청   서울특별시 동작구 신대방동 460-18 (기상청)   126.920722         5   

          lat   spr_temp  season  sum_temp  fal_temp    y    x  win_temp  
0  37.4528634  11.438756  spring       NaN       NaN  NaN  NaN       NaN  
1  37.4603141  12.775027  spring       NaN       NaN  NaN  NaN       NaN  
2  37.4828143  13.478883  spring       NaN       NaN  NaN  NaN       NaN  
3  37.4861128  12.217729  spring       NaN       NaN  NaN  NaN       NaN  
4  37.4963142  12.915244  spring       NaN       NaN  NaN  NaN       NaN  


In [ ]:
import json
import pandas as pd

# 계절별 파일 경로 (경로는 로컬 환경에 맞게 수정)
file_paths = {
    'spring': '/content/weather_spring.json',
    'summer': '/content/weather_summer.json',
    'fall': '/content/weather_fall.json',
    'winter': '/content/weather_winter.json'
}

dfs = []
for season, path in file_paths.items():
    with open(path, encoding='utf-8') as f:
        data = json.load(f)
        df = pd.DataFrame(data['DATA'])

        # 칼럼명 통일 (season별로 다름, temp도 처리)
        if 'aws_cde' in df.columns:
            df.rename(columns={'aws_cde': 'observatory_id'}, inplace=True)
        if 'AWS_CDE' in df.columns:
            df.rename(columns={'AWS_CDE': 'observatory_id'}, inplace=True)
        if 'objectid' in df.columns:
            df.rename(columns={'objectid': 'region_id'}, inplace=True)
        if 'OBJECTID' in df.columns:
            df.rename(columns={'OBJECTID': 'region_id'}, inplace=True)

        # lng, lat 보정
        if 'X' in df.columns:
            df.rename(columns={'X': 'lng'}, inplace=True)
        if 'Y' in df.columns:
            df.rename(columns={'Y': 'lat'}, inplace=True)
        if 'LNG' in df.columns:
            df.rename(columns={'LNG': 'lng'}, inplace=True)
        if 'LAT' in df.columns:
            df.rename(columns={'LAT': 'lat'}, inplace=True)

        # 평균기온 칼럼명 찾기
        temp_col = [c for c in df.columns if '_temp' in c or c.endswith('TEMP') or c.endswith('temp')]
        if temp_col:
            temp_colname = temp_col[0]
        else:
            temp_colname = None

        # season 정보 추가
        df['season'] = season

        # 칼럼 순서 재정렬
        cols = ['observatory_id', 'region_id', 'nam', 'address', 'lng', 'lat']
        if temp_colname:
            cols.append(temp_colname)
        cols.append('season')
        # 없는 칼럼은 무시
        cols = [c for c in cols if c in df.columns]
        df = df[cols]
        dfs.append(df)

# 모든 계절 데이터 통합
df_all = pd.concat(dfs, ignore_index=True)

# to_dict 후 json 저장
result = df_all.to_dict(orient='records')
with open('/content/weather_seasons.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# 일부 결과 출력
print(json.dumps(result[:3], ensure_ascii=False, indent=2))

[
  {
    "observatory_id": 509.0,
    "region_id": 1,
    "nam": "관악",
    "address": "서울특별시 관악구 신림동 산56-1 (서울대학교)",
    "lng": "126.9502188",
    "lat": "37.4528634",
    "spr_temp": 11.438756,
    "season": "spring",
    "sum_temp": NaN,
    "fal_temp": NaN,
    "win_temp": NaN
  },
  {
    "observatory_id": 417.0,
    "region_id": 2,
    "nam": "금천",
    "address": "서울특별시 금천구 독산동 1034 (독산초등학교)",
    "lng": "126.9141209",
    "lat": "37.4603141",
    "spr_temp": 12.775027,
    "season": "spring",
    "sum_temp": NaN,
    "fal_temp": NaN,
    "win_temp": NaN
  },
  {
    "observatory_id": 401.0,
    "region_id": 3,
    "nam": "서초",
    "address": "서울특별시 서초구 서초동 1650 (서울교육대학교)",
    "lng": "127.0179133",
    "lat": "37.4828143",
    "spr_temp": 13.478883,
    "season": "spring",
    "sum_temp": NaN,
    "fal_temp": NaN,
    "win_temp": NaN
  }
]


### seasonal temperature + function

In [ ]:
import pandas as pd

# Load the weather seasons data
df = pd.read_json('weather_seasons.json')

In [ ]:
# Define the categorization function
def categorize(temp):
    if pd.isnull(temp):
        return None
    if 18 <= temp <= 23:
        return 1
    elif 23 < temp <= 26:
        return 0
    else:
        return -1

# Apply the function to each seasonal temperature column
for col in ['spr_temp', 'sum_temp', 'fal_temp', 'win_temp']:
    df[col + '_cat'] = df[col].apply(categorize)

# Save the categorized data
output_path = 'weather_seasons_categorized.json'
df.to_json(output_path, orient='records', force_ascii=False)

print(f"Categorized data saved to {output_path}")

Categorized data saved to weather_seasons_categorized.json


In [ ]:
df

,observatory_id,region_id,nam,address,lng,lat,spr_temp,season,sum_temp,fal_temp,win_temp,spr_temp_cat,sum_temp_cat,fal_temp_cat,win_temp_cat
0,509.0,1,관악,서울특별시 관악구 신림동 산56-1 (서울대학교),126.950219,37.452863,11.438756,spring,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
1,417.0,2,금천,서울특별시 금천구 독산동 1034 (독산초등학교),126.914121,37.460314,12.775027,spring,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
2,401.0,3,서초,서울특별시 서초구 서초동 1650 (서울교육대학교),127.017913,37.482814,13.478883,spring,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
3,423.0,4,구로,서울특별시 구로구 궁동 213-42 (수궁동사무소),126.831229,37.486113,12.217729,spring,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
4,410.0,5,기상청,서울특별시 동작구 신대방동 460-18 (기상청),126.920722,37.496314,12.915244,spring,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,NaN,23,북한산,서울특별시 종로구 구기동 산1 (승가사),126.954419,37.618295,NaN,winter,NaN,NaN,-2.605182,NaN,NaN,NaN,-1.0
104,NaN,24,노원,서울특별시 노원구 공릉동 230-3 (육군사관학교),127.087304,37.622196,NaN,winter,NaN,NaN,-0.941102,NaN,NaN,NaN,-1.0
105,NaN,25,강북,서울특별시 강북구 수유동 192-49 (강북구청 본관),126.999613,37.636093,NaN,winter,NaN,NaN,0.434751,NaN,NaN,NaN,-1.0
106,NaN,26,도봉,서울특별시 도봉구 방학동 310 (신방학초등학교),127.033110,37.666091,NaN,winter,NaN,NaN,-0.652582,NaN,NaN,NaN,-1.0


In [ ]:
uploaded = files.upload()

Saving weather_seasons_categorized.json to weather_seasons_categorized (1).json


### 정리된 버전의 코드

In [ ]:
import json
import pandas as pd
from google.colab import files

# ---------------------------------------------
# 1. 계절별 JSON 파일 업로드
# ---------------------------------------------
print("▶ 봄·여름·가을·겨울 데이터 파일(weather_spring.json, weather_summer.json, weather_fall.json, weather_winter.json)을 모두 선택하여 업로드하세요.")
uploaded = files.upload()  # 4개 파일 선택

# ---------------------------------------------
# 2. 파일명 → 시즌 및 온도컬럼 매핑
# ---------------------------------------------
season_map = {
    'weather_spring.json': ('spring', 'spr_temp'),
    'weather_summer.json': ('summer', 'sum_temp'),
    'weather_fall.json':   ('fall',   'fal_temp'),
    'weather_winter.json': ('winter', 'win_temp')
}

df_list = []
for fname, content in uploaded.items():
    # 매핑 확인
    season, temp_col = season_map.get(fname, (None, None))
    if season is None:
        print(f"⚠️ 알 수 없는 파일: {fname}")
        continue

    # JSON 파싱 및 DataFrame 변환
    data = json.loads(content.decode('utf-8')).get('DATA', [])
    df = pd.json_normalize(data)

    # 시즌 정보 및 공통 컬럼명 부여
    df['season'] = season
    df['avg_temp'] = df[temp_col]

    # 필요 컬럼만 선택
    cols = ['aws_cde', 'nam', 'address', 'lat', 'lng', 'avg_temp', 'season']
    df_list.append(df[cols])

# ---------------------------------------------
# 3. 시즌별 데이터 병합
# ---------------------------------------------
combined = pd.concat(df_list, ignore_index=True)
print(f"▶ 병합 완료: {combined.shape[0]}개 레코드, 컬럼 {combined.columns.tolist()}")

display(combined.head())  # 확인용

# ---------------------------------------------
# 4. 저장 및 다운로드
# ---------------------------------------------
output_csv = 'seasonal_weather_summary.csv'
output_json = 'seasonal_weather_summary.json'

combined.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"✅ '{output_csv}' 저장 완료")

combined.to_json(output_json, orient='records', force_ascii=False, indent=2)
print(f"✅ '{output_json}' 저장 완료")

files.download(output_csv)
files.download(output_json)


## **5-2) 쾌적성_ 미세먼지 데이터**

In [ ]:
from google.colab import files
import io
import pandas as pd
from IPython.display import display

# 1. metro_air.csv 업로드
uploaded = files.upload()
fname = list(uploaded.keys())[0]

# 2. CSV 읽기 (CP949 우선, 실패 시 Latin-1 폴백)
try:
    df_air = pd.read_csv(io.BytesIO(uploaded[fname]), encoding='cp949')
except UnicodeDecodeError:
    df_air = pd.read_csv(io.BytesIO(uploaded[fname]), encoding='latin1')

# 3. 컬럼 목록 및 샘플 출력
print("▶ 파일:", fname)
print("■ 컬럼:", df_air.columns.tolist())
print("■ 레코드 수:", df_air.shape[0])
display(df_air.head())


Saving metro_air.csv to metro_air.csv
▶ 파일: metro_air.csv
■ 컬럼: ['연번', '호선', '역명', '미세먼지', '초미세먼지', '이산화탄소', '폼알데하이드', '일산화탄소', '이산화질소', '라돈', '휘발성유기화합물', '데이터기준일자']
■ 레코드 수: 263


,연번,호선,역명,미세먼지,초미세먼지,이산화탄소,폼알데하이드,일산화탄소,이산화질소,라돈,휘발성유기화합물,데이터기준일자
0,1,1호선,서울역(1),33.7,68.8,525.0,15.8,1.7,0.016,10.1,123.0,2023-12-31
1,2,1호선,시청(1),54.7,81.7,444.0,6.0,1.4,0.017,13.4,71.6,2023-12-31
2,3,1호선,종각,49.8,162.1,605.5,7.0,0.9,0.018,15.8,111.9,2023-12-31
3,4,1호선,종로3가(1),64.7,75.9,575.5,7.0,0.5,0.016,8.7,150.7,2023-12-31
4,5,1호선,종로5가,71.4,106.5,554.0,8.4,0.8,0.017,11.6,105.2,2023-12-31


In [ ]:
display(df_air.tail())

,연번,호선,역명,미세먼지,초미세먼지,이산화탄소,폼알데하이드,일산화탄소,이산화질소,라돈,휘발성유기화합물,데이터기준일자
258,259,9호선,송파나루,41.3,58.4,349.0,21.0,1.3,0.033,30.0,101.7,2023-12-31
259,260,9호선,한성백제,38.1,33.3,310.5,22.3,1.1,0.015,19.3,96.5,2023-12-31
260,261,9호선,올림픽공원,54.8,50.1,372.7,28.5,1.3,0.015,27.3,88.8,2023-12-31
261,262,9호선,둔촌오륜,38.2,49.1,347.0,25.1,1.3,0.035,29.6,74.7,2023-12-31
262,263,9호선,중앙보훈병원,22.3,44.9,378.0,25.6,1.2,0.012,40.4,89.0,2023-12-31


In [ ]:
from google.colab import files
import io
import pandas as pd
import re
from IPython.display import display

# 1. 12개의 Excel 파일 업로드
print("▶ 1~12월 용 Microdust 파일 12개를 모두 선택하여 업로드하세요.")
uploaded = files.upload()


▶ 1~12월 용 Microdust 파일 12개를 모두 선택하여 업로드하세요.


Saving microdust_1.xlsx to microdust_1.xlsx
Saving microdust_2.xlsx to microdust_2.xlsx
Saving microdust_3.xlsx to microdust_3.xlsx
Saving microdust_4.xlsx to microdust_4.xlsx
Saving microdust_5.xlsx to microdust_5.xlsx
Saving microdust_6.xlsx to microdust_6.xlsx
Saving microdust_7.xlsx to microdust_7.xlsx
Saving microdust_8.xlsx to microdust_8.xlsx
Saving microdust_9.xlsx to microdust_9.xlsx
Saving microdust_10.xlsx to microdust_10.xlsx
Saving microdust_11.xlsx to microdust_11.xlsx
Saving microdust_12.xlsx to microdust_12.xlsx


In [ ]:
# 1) openpyxl 설치
!pip install openpyxl

# 2) 설치가 완료되면 아래와 같이 engine 지정(optional) 후 다시 실행
import io, re, pandas as pd
from google.colab import files
from IPython.display import display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
# 여러 파일을 처리한다고 가정
dfs = []
for fname, content in uploaded.items():
    # 월 정보 추출
    month = int(re.search(r'_(\d+)\.xlsx$', fname).group(1))
    # Excel 읽기 (engine 명시)
    df = pd.read_excel(io.BytesIO(content), engine='openpyxl')
    df['월'] = month
    dfs.append(df)


In [ ]:
# 2) 이후 전처리 코드 실행
df_clean = combined.rename(
    columns={'지점명': '역명'}
).drop(
    columns=['운영기관', '위치', '측정월', '측정일자', '일평균']
)

In [ ]:
from IPython.display import display
display(df_clean.head())

,역명,호선,월평균,월
0,청량리,1호선,43.1,1
1,제기동,1호선,66.5,1
2,신설동,1호선,64.7,1
3,동묘,1호선,70.7,1
4,동대문,1호선,84.7,1


In [ ]:
df_clean.to_json('metro_microdust_cleaned.json', orient='records', force_ascii=False, indent=2)
from google.colab import files; files.download('metro_microdust_cleaned.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 3. 데이터프레임 합치기
combined = pd.concat(dfs, ignore_index=True)

In [ ]:
print("■ 합쳐진 DataFrame 크기:", combined.shape)
display(combined.head())

■ 합쳐진 DataFrame 크기: (27555, 9)


,지점명,운영기관,호선,위치,측정월,월평균,월,측정일자,일평균
0,청량리,서울교통공사,1호선,서울,202401.0,43.1,1,NaN,NaN
1,제기동,서울교통공사,1호선,서울,202401.0,66.5,1,NaN,NaN
2,신설동,서울교통공사,1호선,서울,202401.0,64.7,1,NaN,NaN
3,동묘,서울교통공사,1호선,서울,202401.0,70.7,1,NaN,NaN
4,동대문,서울교통공사,1호선,서울,202401.0,84.7,1,NaN,NaN


### 정리된 버전의 코드

In [ ]:
# ---------------------------------------------
# 0. (선택) openpyxl 설치 — 최초 한 번만 실행
# ---------------------------------------------
!pip install openpyxl

# ---------------------------------------------
# 1. 라이브러리 임포트
# ---------------------------------------------
import io
import re
import pandas as pd
from google.colab import files
from IPython.display import display

# ---------------------------------------------
# 2. 1~12월 미세먼지 엑셀 파일 업로드
# ---------------------------------------------
print("▶ 1~12월 microdust_1.xlsx ~ microdust_12.xlsx 파일을 모두 선택하여 업로드하세요.")
uploaded = files.upload()  # 업로드 창에서 12개 파일 선택

# ---------------------------------------------
# 3. 엑셀 파일 월별로 읽어 통합
# ---------------------------------------------
def load_microdust_excels(uploaded_files):
    """
    uploaded_files: files.upload() 결과 dict
    파일명에서 '_<월>.xlsx' 를 추출해 '월' 컬럼을 추가한 뒤
    모든 DataFrame을 concat 해서 반환
    """
    dfs = []
    for fname, content in uploaded_files.items():
        # 파일명에서 월 정보(1~12) 추출
        m = re.search(r'_(\d+)\.xlsx$', fname)
        month = int(m.group(1)) if m else None

        # openpyxl 엔진으로 읽기
        df = pd.read_excel(io.BytesIO(content), engine='openpyxl')
        df['월'] = month
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

combined = load_microdust_excels(uploaded)
print(f"▶ 통합 완료: {len(combined)}개 레코드, 컬럼 {combined.columns.tolist()}")
display(combined.head())

# ---------------------------------------------
# 4. 불필요 컬럼 제거 및 컬럼명 변경
# ---------------------------------------------
# 예시: ['운영기관','위치','측정월','측정일자','일평균'] 컬럼 삭제하고
# '지점명' → '역명' 으로 변경
df_clean = (
    combined
    .rename(columns={'지점명': '역명'})
    .drop(columns=['운영기관', '위치', '측정월', '측정일자', '일평균'], errors='ignore')
)

print("▶ 전처리 후 컬럼:", df_clean.columns.tolist())
display(df_clean.head())

# ---------------------------------------------
# 5. JSON 저장 및 다운로드
# ---------------------------------------------
output_file = 'metro_microdust_cleaned.json'
df_clean.to_json(
    output_file,
    orient='records',
    force_ascii=False,
    indent=2
)
print(f"✅ '{output_file}' 저장 완료")

files.download(output_file)

# **지하철 데이터 합치기**

In [ ]:
import pandas as pd
from google.colab import files

# 1. Upload files interactively
uploaded = files.upload()

Saving metro_people.csv to metro_people.csv
Saving combined_facilities.json to combined_facilities.json
Saving metro_transfer_time.csv to metro_transfer_time.csv
Saving metro-line.json to metro-line.json
Saving metro-station.json to metro-station.json
Saving metro_microdust.json to metro_microdust.json
Saving metro_late.json to metro_late.json
Saving metro_accident_counts.json to metro_accident_counts.json
Saving weather_fall.json to weather_fall.json
Saving weather_spring.json to weather_spring.json
Saving weather_summer.json to weather_summer.json
Saving weather_winter.json to weather_winter.json


In [ ]:
import pandas as pd
from google.colab import files

# 1. 인터랙티브하게 파일 업로드
uploaded = files.upload()  # 업로드 창이 뜹니다

Saving combined_facilities.json to combined_facilities.json
Saving metro_late.json to metro_late.json
Saving metro_accident_counts.json to metro_accident_counts.json
Saving metro-station.json to metro-station.json


In [ ]:
# 1) Load data
stations = pd.read_json('metro-station.json')
accidents = pd.read_json('metro_accident_counts.json')
late = pd.read_json('metro_late.json')
facilities = pd.read_json('combined_facilities.json')

In [ ]:
print(f"   stations: {stations.shape}, accidents: {accidents.shape}, late: {late.shape}, facilities: {facilities.shape}")


   stations: (785, 7), accidents: (297, 2), late: (2056, 4), facilities: (2247, 13)


In [ ]:
# 2) Filter for 1호선~8호선
lines_to_keep = [f"{i}호선" for i in range(1,9)]
stations = stations[stations['line'].isin(lines_to_keep)]
print(f"   필터링 후 stations: {stations.shape}")
print(sorted(stations['line'].unique()))

   필터링 후 stations: (414, 7)
['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선']


In [ ]:
# 3) 코드 기준 역별 노선 정보 그룹화
print("3) station_cd 기준 노선 리스트 생성 중...")
station_lines = stations.groupby('station_cd')['line'] \
    .unique().reset_index().rename(columns={'line':'lines'})
print(f"   station_lines: {station_lines.shape}")

# 3.1) 역명 기준 노선 리스트(物理적 환승역 파악용)
print("3.1) station_name 기준 노선 리스트 생성 중...")
station_lines_by_name = stations.groupby('name')['line'] \
    .unique().reset_index().rename(columns={'line':'lines_by_name','name':'station_name'})
# 3.2) 역명 기준 노선 수 계산
station_lines_by_name['num_lines_by_name'] = station_lines_by_name['lines_by_name'].apply(len)
print("   역명 기준 노선 수 분포:\n", station_lines_by_name['num_lines_by_name'].value_counts())
print("   환승역 예시:\n", station_lines_by_name[station_lines_by_name['num_lines_by_name'] > 1].head())


3) station_cd 기준 노선 리스트 생성 중...
   station_lines: (414, 2)
3.1) station_name 기준 노선 리스트 생성 중...
   역명 기준 노선 수 분포:
 num_lines_by_name
1    326
2     41
3      2
Name: count, dtype: int64
   환승역 예시:
    station_name lines_by_name  num_lines_by_name
1          가락시장    [3호선, 8호선]                  2
2       가산디지털단지    [1호선, 7호선]                  2
14         건대입구    [2호선, 7호선]                  2
20        고속터미널    [3호선, 7호선]                  2
22           공덕    [5호선, 6호선]                  2


In [ ]:
# 4) 기본 station 데이터프레임 생성 및 코드 기준 정보 병합
print("4) 기본 station 데이터프레임 생성 중...")
stations_unique = stations.drop_duplicates(subset='station_cd')[['station_cd','name','lat','lng']]
stations_unique = stations_unique.rename(columns={'name':'station_name'})
stations_unique = stations_unique.merge(station_lines, on='station_cd')
print(f"   stations_unique (code 기준): {stations_unique.shape}")

# 4.1) 역명 기준 노선 리스트 및 환승역 정보 병합
print("4.1) 역명 기준 노선 및 환승역 정보 병합 중...")
stations_unique = stations_unique.merge(
    station_lines_by_name, on='station_name', how='left'
)
print(f"   stations_unique (merged name 기준): {stations_unique.shape}")


4) 기본 station 데이터프레임 생성 중...
   stations_unique (code 기준): (414, 5)
4.1) 역명 기준 노선 및 환승역 정보 병합 중...
   stations_unique (merged name 기준): (414, 7)


In [ ]:
# 5) 사고 횟수 병합
print("5) 사고 횟수 병합 중...")
stations_unique = stations_unique.merge(
    accidents.rename(columns={'역명':'station_name','사고횟수':'accident_count'}),
    on='station_name', how='left'
).fillna({'accident_count': 0})
print("   사고 횟수 예시:\n", stations_unique[['station_name','accident_count']].head())


5) 사고 횟수 병합 중...
   사고 횟수 예시:
   station_name  accident_count
0          망월사             0.0
1          신도림            49.0
2           금정             5.0
3           보산             0.0
4           덕정             0.0


In [ ]:
stations_unique

,station_cd,station_name,lat,lng,lines,lines_by_name,num_lines_by_name,accident_count
0,1904,망월사,37.709914,127.047455,[1호선],[1호선],1,0.0
1,1007,신도림,37.508787,126.891144,[1호선],"[1호선, 2호선]",2,49.0
2,1708,금정,37.372221,126.943429,[1호선],"[1호선, 4호선]",2,5.0
3,1914,보산,37.913702,127.057277,[1호선],[1호선],1,0.0
4,1911,덕정,37.843188,127.061277,[1호선],[1호선],1,0.0
...,...,...,...,...,...,...,...,...
409,2813,강동구청,37.530341,127.120508,[8호선],[8호선],1,0.0
410,2817,송파,37.499703,127.112183,[8호선],[8호선],1,3.0
411,2819,문정,37.485855,127.122500,[8호선],[8호선],1,11.0
412,2821,복정,37.471052,127.126732,[8호선],[8호선],1,2.0


In [ ]:
# 6) 노선별 평균 지연 시간 계산
print("6) 노선별 평균 지연 시간 계산 중...")
delay_avg = late.groupby('노선')['최대지연시간(분)'] \
    .mean().reset_index().rename(columns={'노선':'line','최대지연시간(분)':'avg_delay_minutes'})
print("   노선별 평균 지연시간 예시:\n", delay_avg.head())

6) 노선별 평균 지연 시간 계산 중...
   노선별 평균 지연시간 예시:
   line  avg_delay_minutes
0  1호선          10.369318
1  2호선           9.797844
2  3호선           9.393305
3  4호선          11.704782
4  5호선           8.429752


In [ ]:
# 7) 역별 평균 지연 시간 할당 (물리적 노선 기준)
print("7) 역별 평균 지연 시간 할당 중...")
stations_unique['avg_delay_minutes'] = stations_unique['lines_by_name'].apply(
    lambda lines: delay_avg[delay_avg['line'].isin(lines)]['avg_delay_minutes'].mean()
)
print("   avg_delay_minutes 예시:\n", stations_unique[['station_name','lines_by_name','avg_delay_minutes']].head())


7) 역별 평균 지연 시간 할당 중...
   avg_delay_minutes 예시:
   station_name lines_by_name  avg_delay_minutes
0          망월사         [1호선]          10.369318
1          신도림    [1호선, 2호선]          10.083581
2           금정    [1호선, 4호선]          11.037050
3           보산         [1호선]          10.369318
4           덕정         [1호선]          10.369318


In [ ]:
stations_unique

,station_cd,station_name,lat,lng,lines,lines_by_name,num_lines_by_name,accident_count,avg_delay_minutes
0,1904,망월사,37.709914,127.047455,[1호선],[1호선],1,0.0,10.369318
1,1007,신도림,37.508787,126.891144,[1호선],"[1호선, 2호선]",2,49.0,10.083581
2,1708,금정,37.372221,126.943429,[1호선],"[1호선, 4호선]",2,5.0,11.037050
3,1914,보산,37.913702,127.057277,[1호선],[1호선],1,0.0,10.369318
4,1911,덕정,37.843188,127.061277,[1호선],[1호선],1,0.0,10.369318
...,...,...,...,...,...,...,...,...,...
409,2813,강동구청,37.530341,127.120508,[8호선],[8호선],1,0.0,8.846154
410,2817,송파,37.499703,127.112183,[8호선],[8호선],1,3.0,8.846154
411,2819,문정,37.485855,127.122500,[8호선],[8호선],1,11.0,8.846154
412,2821,복정,37.471052,127.126732,[8호선],[8호선],1,2.0,8.846154


In [ ]:
# 8) 편의시설 정보 처리 및 병합
print("8) 편의시설 정보 처리 및 병합 중...")
# 8.1) 리프트·엘리베이터 개수 합산
lift_sum = facilities.groupby('station_name')['lift_count'] \
    .sum().reset_index().rename(columns={'lift_count':'total_lift_count'})
elev_sum = facilities.groupby('station_name')['elevator_count'] \
    .sum().reset_index().rename(columns={'elevator_count':'total_elevator_count'})

# NaN을 0으로 채우기 (facet: 없는 역은 0 개)
lift_sum['total_lift_count'] = lift_sum['total_lift_count'].fillna(0)
elev_sum['total_elevator_count'] = elev_sum['total_elevator_count'].fillna(0)

# 결과 확인
print(f"  lift_sum shape: {lift_sum.shape}")
print("  lift_sum head:")
print(lift_sum.head(10))
print(f"  elev_sum shape: {elev_sum.shape}")
print("  elev_sum head:")
print(elev_sum.head(10))

8) 편의시설 정보 처리 및 병합 중...
  lift_sum shape: (404, 2)
  lift_sum head:
  station_name  total_lift_count
0     4.19민주묘지               0.0
1         가락시장               0.0
2      가락시장(3)               0.0
3      가락시장(8)               0.0
4      가산디지털단지               0.0
5   가산디지털단지(7)               0.0
6           가양               0.0
7          가오리               0.0
8           가좌               0.0
9           강남               1.0
  elev_sum shape: (404, 2)
  elev_sum head:
  station_name  total_elevator_count
0     4.19민주묘지                   3.0
1         가락시장                   9.0
2      가락시장(3)                   0.0
3      가락시장(8)                   0.0
4      가산디지털단지                   3.0
5   가산디지털단지(7)                   0.0
6           가양                   2.0
7          가오리                   2.0
8           가좌                   4.0
9           강남                   6.0


In [ ]:
# 8.2) 화장실 접근성
print("8.2) 화장실 접근성 처리 중...")
gate = facilities.groupby('station_name')['gate_access'] \
    .apply(lambda x: '/'.join(sorted(set(x.dropna()))) if len(x.dropna()) > 0 else '')
gate = gate.reset_index().rename(columns={'gate_access':'toilet_gate_access'})
# 빈 문자열과 NaN 처리
gate['toilet_gate_access'] = gate['toilet_gate_access'].replace('', '없음').fillna('없음')
print(gate.head())

# 8.3) 화장실 CCTV
print("8.3) 화장실 CCTV 처리 중...")
cctv = facilities.groupby('station_name')['cctv_at_entrance'] \
    .apply(lambda x: 'Y' if 'Y' in set(x.dropna()) else 'N')
cctv = cctv.reset_index().rename(columns={'cctv_at_entrance':'toilet_cctv_at_entrancefh'})
print(cctv.head())

# 8.4) 리모델링 연도(최신)
print("8.4) 리모델링 연도 처리 중...")
remodel = facilities.groupby('station_name')['remodel_year'] \
    .max().reset_index().rename(columns={'remodel_year':'toilet_remodel_yearfh'})
# NaN 처리
remodel['toilet_remodel_yearfh'] = remodel['toilet_remodel_yearfh'].fillna('없음')
print(remodel.head())


8.2) 화장실 접근성 처리 중...
  station_name toilet_gate_access
0     4.19민주묘지                 없음
1         가락시장              내부/외부
2      가락시장(3)                 없음
3      가락시장(8)                 없음
4      가산디지털단지                 외부
8.3) 화장실 CCTV 처리 중...
  station_name toilet_cctv_at_entrancefh
0     4.19민주묘지                         N
1         가락시장                         Y
2      가락시장(3)                         N
3      가락시장(8)                         N
4      가산디지털단지                         Y
8.4) 리모델링 연도 처리 중...
  station_name toilet_remodel_yearfh
0     4.19민주묘지                    없음
1         가락시장                  2015
2      가락시장(3)                    없음
3      가락시장(8)                    없음
4      가산디지털단지                  2019


In [ ]:
# 8.5) stations_unique에 순차적으로 병합
stations_unique = stations_unique \
    .merge(lift_sum, on='station_name', how='left') \
    .merge(elev_sum, on='station_name', how='left') \
    .merge(gate, on='station_name', how='left') \
    .merge(cctv, on='station_name', how='left') \
    .merge(remodel, on='station_name', how='left')

In [ ]:
# NaN 처리
stations_unique['total_lift_count'] = stations_unique['total_lift_count'].fillna(0)
stations_unique['total_elevator_count'] = stations_unique['total_elevator_count'].fillna(0)

print("  병합 후 stations_unique head:")
print(stations_unique[['station_name','total_lift_count','total_elevator_count',
                       'toilet_gate_access','toilet_cctv_at_entrancefh',
                       'toilet_remodel_yearfh']])

  병합 후 stations_unique head:
    station_name  total_lift_count  total_elevator_count toilet_gate_access  \
0            망월사               0.0                   0.0                NaN   
1            신도림               0.0                   6.0              내부/외부   
2             금정               0.0                   0.0                NaN   
3             보산               0.0                   0.0                NaN   
4             덕정               0.0                   0.0                NaN   
..           ...               ...                   ...                ...   
409         강동구청               5.0                   5.0                 외부   
410           송파               0.0                   4.0                 외부   
411           문정               0.0                  16.0                 외부   
412           복정               1.0                   0.0                 외부   
413           장지               0.0                  16.0                 외부   

    toilet_cctv_at_ent

In [ ]:
import pandas as pd
import networkx as nx
import re

In [ ]:
section_max = (
    facilities
    .dropna(subset=['section', 'escalator_length'])
    .groupby(['station_name', 'section'])['escalator_length']
    .max()
    .reset_index()
)

# Helper to convert floor labels to numeric levels
def floor_to_level(floor):
    m = re.match(r'^B(\d+)$', floor)
    if m:
        return -int(m.group(1))
    m = re.match(r'^(\d+)F$', floor)
    if m:
        return int(m.group(1))
    return None

optimal_lengths = []

for station, df in section_max.groupby('station_name'):
    # Determine unique levels present
    levels = set()
    edges = []
    for _, row in df.iterrows():
        # split only first hyphen
        parts = row['section'].split('-', 1)
        if len(parts) != 2:
            continue
        frm, to = parts
        lvl_from = floor_to_level(frm)
        lvl_to = floor_to_level(to)
        if lvl_from is None or lvl_to is None:
            continue
        levels.update([lvl_from, lvl_to])
        weight = row['escalator_length']
        # add bidirectional edges with weight
        edges.append((lvl_from, lvl_to, weight))
        edges.append((lvl_to, lvl_from, weight))
    if not levels:
        optimal_lengths.append((station, 0.0))
        continue
    max_lvl = max(levels)
    min_lvl = min(levels)
    full_levels = list(range(min_lvl, max_lvl + 1))
    G = nx.DiGraph()
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)
    # Add missing adjacent edges with weight=0
    for lvl in full_levels:
        for nxt in [lvl - 1, lvl + 1]:
            if nxt in full_levels and not G.has_edge(lvl, nxt):
                G.add_edge(lvl, nxt, weight=0.0)
    # Compute longest path
    best = 0.0
    try:
        for path in nx.all_simple_paths(G, source=max_lvl, target=min_lvl):
            length = sum(G[u][v]['weight'] for u, v in zip(path, path[1:]))
            best = max(best, length)
    except nx.NetworkXNoPath:
        best = 0.0
    optimal_lengths.append((station, best))

optimal_df = pd.DataFrame(optimal_lengths, columns=['station_name', 'escalator_optimal_length'])

In [ ]:
optimal_df

,station_name,escalator_optimal_length
0,가락시장(3),58.6
1,가락시장(8),55.6
2,가산디지털단지(7),53.4
3,강남구청(7),15.2
4,강동,53.4
...,...,...
247,홍대입구(2),29.4
248,화곡,25.3
249,화랑대,29.6
250,회현,20.4


In [ ]:
# JSON 파일로 저장
optimal_df.to_json("optimal_escalator_data.json", orient="records", force_ascii=False)

# 다운로드 링크 제공
from google.colab import files
files.download("optimal_escalator_data.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(optimal_df.to_string(index=False))


station_name  escalator_optimal_length
     가락시장(3)                    58.600
     가락시장(8)                    55.600
  가산디지털단지(7)                    53.400
     강남구청(7)                    15.200
          강동                    53.400
        강동구청                    12.600
          강변                    14.200
          강일                    51.400
          개롱                    23.600
          거여                    19.900
     건대입구(2)                    12.400
     건대입구(7)                    88.200
         경복궁                    35.400
        경찰병원                    60.800
          고덕                    28.400
         고려대                    57.080
    고속터미널(3)                    22.400
    고속터미널(7)                    31.400
       공덕(5)                    55.400
       공덕(6)                    36.440
          공릉                    61.200
         광나루                    19.600
       광명사거리                    17.200
         광화문                    42.000
         광흥창             

In [ ]:
# 9) 에스컬레이터 최적 길이 병합
print("9) 에스컬레이터 최적 길이 병합 및 결측 확인 중...")
stations_unique = stations_unique.merge(
    optimal_df,
    on='station_name',
    how='left'
)

# 결측값(=엘리베이터 데이터가 아예 없는 역)은 0으로 채우기
stations_unique['escalator_optimal_length'] = stations_unique['escalator_optimal_length'].fillna(0)

# 병합된 통계 보기
print(" * 최적 길이 통계 요약")
print(stations_unique['escalator_optimal_length'].describe())

# 실제 0이 아닌(=병합된) 역 수 확인
num_nonzero = (stations_unique['escalator_optimal_length'] > 0).sum()
print(f" * 에스컬레이터 정보가 있는 역: {num_nonzero}개")
print(f" * 에스컬레이터 정보가 없는 역: {414 - num_nonzero}개")

# 혹시 이름 매칭 누락이 의심된다면, 양쪽에만 있는 역명을 확인
only_in_opt = set(optimal_df['station_name']) - set(stations_unique['station_name'])
only_in_base = set(stations_unique['station_name']) - set(optimal_df['station_name'])
print("▶ optimal_df에는 있는데 stations_unique에 없는 역:", only_in_opt)
print("▶ stations_unique에는 있는데 optimal_df에 없는 역:", list(only_in_base)[:10], "…")


9) 에스컬레이터 최적 길이 병합 및 결측 확인 중...


MergeError: Passing 'suffixes' which cause duplicate columns {'escalator_optimal_length_x'} is not allowed.

In [ ]:
stations_unique

,station_cd,station_name,lat,lng,lines,lines_by_name,num_lines_by_name,accident_count,avg_delay_minutes,total_lift_count,total_elevator_count,toilet_gate_access,toilet_cctv_at_entrancefh,toilet_remodel_yearfh,escalator_optimal_length_x,escalator_optimal_length_y,escalator_optimal_length
0,1904,망월사,37.709914,127.047455,[1호선],[1호선],1,0.0,10.369318,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
1,1007,신도림,37.508787,126.891144,[1호선],"[1호선, 2호선]",2,49.0,10.083581,0.0,6.0,내부/외부,Y,2011,0.0,NaN,0.0
2,1708,금정,37.372221,126.943429,[1호선],"[1호선, 4호선]",2,5.0,11.037050,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
3,1914,보산,37.913702,127.057277,[1호선],[1호선],1,0.0,10.369318,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
4,1911,덕정,37.843188,127.061277,[1호선],[1호선],1,0.0,10.369318,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,2813,강동구청,37.530341,127.120508,[8호선],[8호선],1,0.0,8.846154,5.0,5.0,외부,Y,2021,12.6,12.6,12.6
410,2817,송파,37.499703,127.112183,[8호선],[8호선],1,3.0,8.846154,0.0,4.0,외부,Y,2015,21.8,21.8,21.8
411,2819,문정,37.485855,127.122500,[8호선],[8호선],1,11.0,8.846154,0.0,16.0,외부,Y,2023,24.8,24.8,24.8
412,2821,복정,37.471052,127.126732,[8호선],[8호선],1,2.0,8.846154,1.0,0.0,외부,Y,2015,0.0,NaN,0.0


In [ ]:
# 1. 인터랙티브하게 파일 업로드
uploaded = files.upload()  # 업로드 창이 뜹니다.

file_path = "metro_depth.csv"

# 2. 다양한 인코딩 시도 (주로 한글 CSV는 cp949 또는 euc-kr일 수 있음)
try:
    df_depth = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df_depth = pd.read_csv(file_path, encoding='cp949')  # 또는 'euc-kr'

# 3. 컬럼명 출력
print("📌 컬럼 목록:")
print(df_depth.columns.tolist())

# 4. 데이터 미리 보기
df_depth.head(10
)

Saving metro_depth.csv to metro_depth.csv
📌 컬럼 목록:
['연번', '호선', '역명', '층수', '형식', '지반고', '레일면고', '선로기준정거장깊이', '정거장깊이', '비고']


,연번,호선,역명,층수,형식,지반고,레일면고,선로기준정거장깊이,정거장깊이,비고
0,1,1,서울,B2,섬식,129.99,117.04,12.95,11.85,"4호선,경의중앙선,공항철도환승"
1,2,1,시청,B2,상대식,129.97,118.82,11.15,10.05,2호선환승
2,3,1,종각,B2,상대식,128.77,116.24,12.53,11.43,NaN
3,4,1,종로3가,B2,상대식,124.38,112.04,12.34,11.24,"3,5호선환승"
4,5,1,종로5가,B2,상대식,121.75,109.26,12.49,11.39,NaN
5,6,1,동대문,B2,상대식,120.52,108.93,11.59,10.49,4호선환승
6,7,1,동묘앞,B1,상대식,119.91,109.97,9.94,8.84,6호선환승
7,8,1,신설동,B2,상대식,118,108.46,9.54,8.44,"2호선(지선),우이신설선환승"
8,9,1,제기동,B2,상대식,115.91,105.49,10.42,9.32,NaN
9,10,1,청량리,B2,섬식,114.41,107.97,6.44,5.34,"분당선,경의중앙선,경춘선환승"


In [ ]:
# 9) Metro Depth 데이터 전처리 (depth_sel 생성부)
depth_sel = (
    depth[['역명','층수','지반고','비고']]  # '호선' 컬럼을 뺐습니다
    .rename(columns={
        '역명':'station_name',
        '층수':'num_levels',
        '지반고':'ground_elevation',
        '비고':'transfer_info'
    })
    .drop_duplicates('station_name')
)


print(f"depth_sel shape: {depth_sel.shape}")
print(depth_sel.head(10))

depth_sel shape: (244, 4)
  station_name num_levels ground_elevation     transfer_info
0           서울         B2           129.99  4호선,경의중앙선,공항철도환승
1           시청         B2           129.97             2호선환승
2           종각         B2           128.77               NaN
3         종로3가         B2           124.38           3,5호선환승
4         종로5가         B2           121.75               NaN
5          동대문         B2           120.52             4호선환승
6          동묘앞         B1           119.91             6호선환승
7          신설동         B2              118   2호선(지선),우이신설선환승
8          제기동         B2           115.91               NaN
9          청량리         B2           114.41   분당선,경의중앙선,경춘선환승


In [ ]:
stations_unique

,station_cd,station_name,lat,lng,lines,lines_by_name,num_lines_by_name,accident_count,avg_delay_minutes,total_lift_count,...,toilet_remodel_yearfh,escalator_optimal_length_x,escalator_optimal_length_y,escalator_optimal_length,num_levels_x,ground_elevation_x,transfer_info_x,num_levels_y,ground_elevation_y,transfer_info_y
0,1904,망월사,37.709914,127.047455,[1호선],[1호선],1,0.0,10.369318,0.0,...,NaN,0.0,NaN,0.0,,,,NaN,NaN,NaN
1,1007,신도림,37.508787,126.891144,[1호선],"[1호선, 2호선]",2,49.0,10.083581,0.0,...,2011,0.0,NaN,0.0,B2,111.52,2호선(지선).국철환승,B2,111.52,2호선(지선).국철환승
2,1708,금정,37.372221,126.943429,[1호선],"[1호선, 4호선]",2,5.0,11.037050,0.0,...,NaN,0.0,NaN,0.0,,,,NaN,NaN,NaN
3,1914,보산,37.913702,127.057277,[1호선],[1호선],1,0.0,10.369318,0.0,...,NaN,0.0,NaN,0.0,,,,NaN,NaN,NaN
4,1911,덕정,37.843188,127.061277,[1호선],[1호선],1,0.0,10.369318,0.0,...,NaN,0.0,NaN,0.0,,,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,2813,강동구청,37.530341,127.120508,[8호선],[8호선],1,0.0,8.846154,5.0,...,2021,12.6,12.6,12.6,B3,116.92,,B3,116.92,NaN
410,2817,송파,37.499703,127.112183,[8호선],[8호선],1,3.0,8.846154,0.0,...,2015,21.8,21.8,21.8,B2,116.24,,B2,116.24,NaN
411,2819,문정,37.485855,127.122500,[8호선],[8호선],1,11.0,8.846154,0.0,...,2023,24.8,24.8,24.8,B2,117.88,,B2,117.88,NaN
412,2821,복정,37.471052,127.126732,[8호선],[8호선],1,2.0,8.846154,1.0,...,2015,0.0,NaN,0.0,B3,118.42,분당선환승,B3,118.42,분당선환승


In [ ]:
# 컬럼명 확인
print("Columns:", stations_unique.columns.tolist())

Columns: ['station_cd', 'station_name', 'lat', 'lng', 'lines', 'lines_by_name', 'num_lines_by_name', 'accident_count', 'avg_delay_minutes', 'total_lift_count', 'total_elevator_count', 'toilet_gate_access', 'toilet_cctv_at_entrancefh', 'toilet_remodel_yearfh', 'escalator_optimal_length_x', 'escalator_optimal_length_y', 'escalator_optimal_length', 'num_levels_x', 'ground_elevation_x', 'transfer_info_x', 'num_levels_y', 'ground_elevation_y', 'transfer_info_y']


In [ ]:
# stations_unique가 이미 최종 테이블이라 가정
print("Before drop:", stations_unique.shape)

Before drop: (414, 23)


In [ ]:
stations_unique = stations_unique[stations_unique['num_levels_x'] != '']
print("After drop:", stations_unique.shape)

After drop: (274, 23)


In [ ]:
# 2) 컬럼명 정리: _x/_y 접미사 제거
stations_unique = stations_unique.rename(columns={
    'num_levels_x': 'num_levels',
    'ground_elevation_x': 'ground_elevation',
    'transfer_info_x': 'transfer_info'
})


In [ ]:
# (만약 y 버전 접미사 컬럼이 남아 있다면, drop하세요)
for c in stations_unique.columns:
    if c.endswith('_y'):
        stations_unique = stations_unique.drop(columns=[c])

# 결과 확인
print(stations_unique[['station_name','num_levels','ground_elevation','transfer_info']].head())

   station_name num_levels ground_elevation transfer_info
1           신도림         B2           111.52  2호선(지선).국철환승
36          동묘앞         B1           119.91         6호선환승
42           창동         고가           122.99          국철환승
45      가산디지털단지         B4           109.59         1호선환승
52           신길         B4            108.9         1호선환승


In [ ]:
print("total:", stations_unique.shape)

total: (274, 19)


In [ ]:
stations_unique

,station_cd,station_name,lat,lng,lines,lines_by_name,num_lines_by_name,accident_count,avg_delay_minutes,total_lift_count,total_elevator_count,toilet_gate_access,toilet_cctv_at_entrancefh,toilet_remodel_yearfh,escalator_optimal_length_x,escalator_optimal_length,num_levels,ground_elevation,transfer_info
1,1007,신도림,37.508787,126.891144,[1호선],"[1호선, 2호선]",2,49.0,10.083581,0.0,6.0,내부/외부,Y,2011,0.000,0.000,B2,111.52,2호선(지선).국철환승
36,0159,동묘앞,37.573197,127.016480,[1호선],"[1호선, 6호선]",2,11.0,9.629104,25.0,10.0,내부/외부,Y,2020,0.000,0.000,B1,119.91,6호선환승
42,1022,창동,37.653007,127.047806,[1호선],"[1호선, 4호선]",2,28.0,11.037050,0.0,3.0,내부,Y,2009,0.000,0.000,고가,122.99,국철환승
45,1702,가산디지털단지,37.481581,126.882581,[1호선],"[1호선, 7호선]",2,12.0,9.586573,0.0,3.0,외부,Y,2019,0.000,0.000,B4,109.59,1호선환승
52,1032,신길,37.516862,126.917865,[1호선],"[1호선, 5호선]",2,11.0,9.399535,0.0,0.0,외부,Y,2015,74.911,74.911,B4,108.9,1호선환승
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,2813,강동구청,37.530341,127.120508,[8호선],[8호선],1,0.0,8.846154,5.0,5.0,외부,Y,2021,12.600,12.600,B3,116.92,
410,2817,송파,37.499703,127.112183,[8호선],[8호선],1,3.0,8.846154,0.0,4.0,외부,Y,2015,21.800,21.800,B2,116.24,
411,2819,문정,37.485855,127.122500,[8호선],[8호선],1,11.0,8.846154,0.0,16.0,외부,Y,2023,24.800,24.800,B2,117.88,
412,2821,복정,37.471052,127.126732,[8호선],[8호선],1,2.0,8.846154,1.0,0.0,외부,Y,2015,0.000,0.000,B3,118.42,분당선환승


In [ ]:
# UTF-8 한글 깨짐 방지를 위해 force_ascii=False, records 형식으로 저장
stations_unique.to_json('stations_unique.json', orient='records', force_ascii=False, indent=2)


In [ ]:
from google.colab import files
files.download('stations_unique.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

station_cd: 고유 역 코드
station_name: 역명
lines: station_cd 기준 노선 리스트 (대부분 길이 1)
lines_by_name: 역명 기준 노선 리스트 (환승역은 길이 ≥2)
num_lines_by_name: 역명 기준 노선 수
accident_count: 사고 횟수
avg_delay_minutes: 물리적 노선 기준 평균 지연시간
total_lift_count, total_elevator_count: 승강기/엘리베이터 개수
toilet_gate_access, toilet_cctv_at_entrancefh, toilet_remodel_yearfh: 화장실 정보
escalator_length_up, escalator_length_down: 에스컬레이터 방향별 길이

In [ ]:
from google.colab import files
print("필요 파일 업로드")
uploaded = files.upload()

필요 파일 업로드


Saving metro_late.csv to metro_late.csv
Saving bus_accident.csv to bus_accident.csv
Saving bus_late.csv to bus_late.csv
Saving metro_accident_counts.json to metro_accident_counts.json


In [ ]:
from google.colab import files
print("필요 파일 업로드")
uploaded = files.upload()

필요 파일 업로드


Saving bus_late_cleaned (2).xlsx to bus_late_cleaned (2).xlsx


In [ ]:
import pandas as pd

# Load datasets
bus_accident = pd.read_csv('bus_accident.csv', encoding='utf-8')
bus_late = pd.read_csv('bus_late.csv', encoding='utf-8')
bus_late2 = pd.read_excel('bus_late_cleaned (2).xlsx', encoding='utf-8')

TypeError: read_excel() got an unexpected keyword argument 'encoding'

In [ ]:
bus_accident

,﻿기준일,노선ID,버스ID,사고상황코드,제공일시,PART_KEY,노선명,차량번호,좌표X,좌표Y,구간ID,구간명,구간거리,링크ID,링크거리,노드ID,선형좌표일련번호,운행거리
0,﻿20250610,﻿100100001,﻿106024501,﻿4,﻿20250610113407,﻿5,﻿01A,﻿서울74사6175,﻿126.979078,﻿37.576109,﻿100602934,﻿경복궁~경복궁_율곡로_진입,﻿49.0,﻿1000012203,﻿33.33,﻿1000019400,﻿73,﻿0.0
1,﻿20250610,﻿100100009,﻿108032029,﻿3,﻿20250610093801,﻿5,﻿104,﻿서울74사9420,﻿127.022016,﻿37.600707,﻿107603378,﻿이북오도신문사~복선교,﻿116.0,﻿1070039300,﻿205.35,﻿1070018800,﻿174,﻿0.0
2,﻿20250610,﻿100100009,﻿108032029,﻿4,﻿20250610053202,﻿5,﻿104,﻿서울74사9420,﻿126.996654,﻿37.579082,﻿100602747,﻿창경궁.서울대학교병원~원남빌딩,﻿184.0,﻿1000015503,﻿755.46,﻿1000021300,﻿65,﻿0.0
3,﻿20250610,﻿100100010,﻿110058126,﻿4,﻿20250610144605,﻿5,﻿105,﻿서울74사9634,﻿127.063532,﻿37.659573,﻿110603238,﻿상계주공7단지.광림교회앞~7단지영업소,﻿172.0,﻿1100031500,﻿363.44,﻿110000694,﻿322,﻿0.0
4,﻿20250610,﻿100100014,﻿108045531,﻿4,﻿20250610143037,﻿5,﻿109,﻿서울71사1263,﻿127.012901,﻿37.650947,﻿109602312,﻿덕성여대입구~월드아파트,﻿140.0,﻿1080013002,﻿596.77,﻿1080018400,﻿191,﻿0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,﻿20250611,﻿122000001,﻿122014123,﻿4,﻿20250611200217,﻿5,﻿4435,﻿서울70사9361,﻿127.032737,﻿37.474718,﻿121900457,﻿우성아파트.양재초등학교~현대빌라,﻿88.0,﻿1210010502,﻿818.53,﻿121000198,﻿125,﻿0.0
1459,﻿20250611,﻿122000001,﻿122014123,﻿4,﻿20250611200219,﻿5,﻿4435,﻿서울70사9361,﻿127.03274,﻿37.474712,﻿121900457,﻿우성아파트.양재초등학교~현대빌라,﻿88.0,﻿1210010502,﻿818.53,﻿121000198,﻿125,﻿0.0
1460,﻿20250611,﻿123000010,﻿123060310,﻿4,﻿20250611071935,﻿5,﻿741,﻿서울74사7918,﻿127.005855,﻿37.536251,﻿102600284,﻿순천향대학병원~UN한국대표부,﻿0.0,﻿1020013402,﻿314.42,﻿102000068,﻿263,﻿0.0
1461,﻿20250611,﻿123000010,﻿123060310,﻿4,﻿20250611074631,﻿5,﻿741,﻿서울74사7918,﻿126.950007,﻿37.583118,﻿112900162,﻿무악재역~인왕산.홍제원아파트,﻿216.0,﻿1120010604,﻿1142.84,﻿112000400,﻿314,﻿0.0


In [ ]:
# Clean '기준일' column name and values (remove BOM)
orig_col = [c for c in bus_accident.columns if '기준일' in c][0]
bus_accident.rename(columns={orig_col: '기준일'}, inplace=True)
bus_accident['기준일'] = bus_accident['기준일'].astype(str).str.lstrip('\ufeff')

# Parse to datetime
bus_accident['기준일'] = pd.to_datetime(bus_accident['기준일'], format='%Y%m%d')

# Season mapping function
def get_season(month):
    if month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'
    else:
        return 'winter'

bus_accident['season'] = bus_accident['기준일'].dt.month.apply(get_season)


In [ ]:
bus_accident

,기준일,노선ID,버스ID,사고상황코드,제공일시,PART_KEY,노선명,차량번호,좌표X,좌표Y,구간ID,구간명,구간거리,링크ID,링크거리,노드ID,선형좌표일련번호,운행거리,season
0,2025-06-10,﻿100100001,﻿106024501,﻿4,﻿20250610113407,﻿5,﻿01A,﻿서울74사6175,﻿126.979078,﻿37.576109,﻿100602934,﻿경복궁~경복궁_율곡로_진입,﻿49.0,﻿1000012203,﻿33.33,﻿1000019400,﻿73,﻿0.0,여름
1,2025-06-10,﻿100100009,﻿108032029,﻿3,﻿20250610093801,﻿5,﻿104,﻿서울74사9420,﻿127.022016,﻿37.600707,﻿107603378,﻿이북오도신문사~복선교,﻿116.0,﻿1070039300,﻿205.35,﻿1070018800,﻿174,﻿0.0,여름
2,2025-06-10,﻿100100009,﻿108032029,﻿4,﻿20250610053202,﻿5,﻿104,﻿서울74사9420,﻿126.996654,﻿37.579082,﻿100602747,﻿창경궁.서울대학교병원~원남빌딩,﻿184.0,﻿1000015503,﻿755.46,﻿1000021300,﻿65,﻿0.0,여름
3,2025-06-10,﻿100100010,﻿110058126,﻿4,﻿20250610144605,﻿5,﻿105,﻿서울74사9634,﻿127.063532,﻿37.659573,﻿110603238,﻿상계주공7단지.광림교회앞~7단지영업소,﻿172.0,﻿1100031500,﻿363.44,﻿110000694,﻿322,﻿0.0,여름
4,2025-06-10,﻿100100014,﻿108045531,﻿4,﻿20250610143037,﻿5,﻿109,﻿서울71사1263,﻿127.012901,﻿37.650947,﻿109602312,﻿덕성여대입구~월드아파트,﻿140.0,﻿1080013002,﻿596.77,﻿1080018400,﻿191,﻿0.0,여름
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,2025-06-11,﻿122000001,﻿122014123,﻿4,﻿20250611200217,﻿5,﻿4435,﻿서울70사9361,﻿127.032737,﻿37.474718,﻿121900457,﻿우성아파트.양재초등학교~현대빌라,﻿88.0,﻿1210010502,﻿818.53,﻿121000198,﻿125,﻿0.0,여름
1459,2025-06-11,﻿122000001,﻿122014123,﻿4,﻿20250611200219,﻿5,﻿4435,﻿서울70사9361,﻿127.03274,﻿37.474712,﻿121900457,﻿우성아파트.양재초등학교~현대빌라,﻿88.0,﻿1210010502,﻿818.53,﻿121000198,﻿125,﻿0.0,여름
1460,2025-06-11,﻿123000010,﻿123060310,﻿4,﻿20250611071935,﻿5,﻿741,﻿서울74사7918,﻿127.005855,﻿37.536251,﻿102600284,﻿순천향대학병원~UN한국대표부,﻿0.0,﻿1020013402,﻿314.42,﻿102000068,﻿263,﻿0.0,여름
1461,2025-06-11,﻿123000010,﻿123060310,﻿4,﻿20250611074631,﻿5,﻿741,﻿서울74사7918,﻿126.950007,﻿37.583118,﻿112900162,﻿무악재역~인왕산.홍제원아파트,﻿216.0,﻿1120010604,﻿1142.84,﻿112000400,﻿314,﻿0.0,여름


In [ ]:
# 2. Drop unnecessary columns
cols_to_drop = [
    '기준일', '사고상황코드', '제공일시', 'PART_KEY',
    '차량번호', '링크ID', '링크거리', '노드ID','구간ID','구간거리','노선ID',
    '선형좌표일련번호', '운행거리'
]
# Only drop columns that actually exist
existing_drops = [c for c in cols_to_drop if c in bus_accident.columns]
bus_accident.drop(columns=existing_drops, inplace=True)

In [ ]:
bus_accident

,﻿기준일,버스ID,노선명,좌표X,좌표Y,구간명
0,﻿20250610,﻿106024501,﻿01A,﻿126.979078,﻿37.576109,﻿경복궁~경복궁_율곡로_진입
1,﻿20250610,﻿108032029,﻿104,﻿127.022016,﻿37.600707,﻿이북오도신문사~복선교
2,﻿20250610,﻿108032029,﻿104,﻿126.996654,﻿37.579082,﻿창경궁.서울대학교병원~원남빌딩
3,﻿20250610,﻿110058126,﻿105,﻿127.063532,﻿37.659573,﻿상계주공7단지.광림교회앞~7단지영업소
4,﻿20250610,﻿108045531,﻿109,﻿127.012901,﻿37.650947,﻿덕성여대입구~월드아파트
...,...,...,...,...,...,...
1458,﻿20250611,﻿122014123,﻿4435,﻿127.032737,﻿37.474718,﻿우성아파트.양재초등학교~현대빌라
1459,﻿20250611,﻿122014123,﻿4435,﻿127.03274,﻿37.474712,﻿우성아파트.양재초등학교~현대빌라
1460,﻿20250611,﻿123060310,﻿741,﻿127.005855,﻿37.536251,﻿순천향대학병원~UN한국대표부
1461,﻿20250611,﻿123060310,﻿741,﻿126.950007,﻿37.583118,﻿무악재역~인왕산.홍제원아파트


In [ ]:
# Save to Excel
output_path = 'bus_accident.xlsx'
bus_accident.to_excel(output_path, index=False)

In [ ]:
bus_late

,﻿기준일,PART_KEY,매체유형,노선ID,정류장ID,노드순번,정류장명,정류장번호,정류장유형,노선명,...,지수평활2분이하건수.1,지수평활2분초과건수.1,퓨전2분이하건수,퓨전2분초과건수,퓨전음수오차누계,퓨전양수오차누계,평균보정계수계수,신뢰도보정계수계수,지수평활보정계수계수,퓨전보정계수계수
0,﻿20250611,﻿0,﻿1,﻿100000017,﻿100000400,﻿5,﻿청와대,﻿01918,﻿0,﻿TOUR11,...,﻿18,﻿82,﻿100,﻿0,﻿49,﻿51,﻿17,﻿17,﻿17,﻿17
1,﻿20250611,﻿0,﻿1,﻿100000017,﻿100000401,﻿6,﻿통인시장,﻿01910,﻿0,﻿TOUR11,...,﻿50,﻿83,﻿133,﻿0,﻿50,﻿83,﻿40,﻿40,﻿40,﻿40
2,﻿20250611,﻿0,﻿1,﻿100000017,﻿100000402,﻿7,﻿세종문화회관,﻿01911,﻿0,﻿TOUR11,...,﻿110,﻿86,﻿106,﻿90,﻿93,﻿103,﻿46,﻿46,﻿46,﻿46
3,﻿20250611,﻿0,﻿1,﻿100000017,﻿100000403,﻿13,﻿인사동.탑골공원,﻿01913,﻿0,﻿TOUR11,...,﻿38,﻿162,﻿146,﻿54,﻿40,﻿160,﻿33,﻿33,﻿33,﻿33
4,﻿20250611,﻿0,﻿1,﻿100000017,﻿100000404,﻿14,﻿종묘.세운상가,﻿01914,﻿0,﻿TOUR11,...,﻿50,﻿150,﻿121,﻿79,﻿50,﻿150,﻿32,﻿32,﻿32,﻿32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246701,﻿20250610,﻿0,﻿1,﻿241461015,﻿232000433,﻿12,﻿아라대교,﻿41007,﻿0,﻿김포16A,...,﻿1,﻿77,﻿57,﻿21,﻿6,﻿72,﻿16,﻿16,﻿16,﻿16
246702,﻿20250610,﻿0,﻿1,﻿241461015,﻿232000841,﻿6,﻿대우아파트,﻿35788,﻿0,﻿김포16A,...,﻿13,﻿10,﻿23,﻿0,﻿18,﻿5,﻿15,﻿15,﻿15,﻿15
246703,﻿20250610,﻿0,﻿1,﻿241461015,﻿232000842,﻿7,﻿우방아이유쉘,﻿35789,﻿5,﻿김포16A,...,﻿17,﻿10,﻿27,﻿0,﻿19,﻿8,﻿20,﻿20,﻿20,﻿20
246704,﻿20250610,﻿0,﻿1,﻿241461015,﻿232000852,﻿4,﻿고촌근린공원,﻿35799,﻿0,﻿김포16A,...,﻿9,﻿5,﻿14,﻿0,﻿9,﻿5,﻿9,﻿9,﻿9,﻿9


In [ ]:
# 2. Clean '기준일' column name and values (remove BOM if present)
orig_col = [c for c in bus_late.columns if '기준일' in c][0]
bus_late.rename(columns={orig_col: '기준일'}, inplace=True)
bus_late['기준일'] = bus_late['기준일'].astype(str).str.lstrip('\ufeff')

# 3. Parse to datetime
bus_late['기준일'] = pd.to_datetime(bus_late['기준일'], format='%Y%m%d')

# 4. Season mapping function
def get_season(month):
    if month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'
    else:
        return 'winter'

bus_late['season'] = bus_late['기준일'].dt.month.apply(get_season)


In [ ]:
# 1. Drop specified columns if they exist
cols_to_drop = ['기준일', 'PART_KEY', '매체유형','정류장유형','노선유형',  '퓨전음수오차누계', '퓨전양수오차누계',
    '평균보정계수계수', '신뢰도보정계수계수',
    '지수평활보정계수계수', '퓨전보정계수계수']
existing_drops = [c for c in cols_to_drop if c in bus_late.columns]
bus_late_cleaned = bus_late.drop(columns=existing_drops)


In [ ]:
# Save to Excel
output_path = 'bus_late_cleaned.xlsx'
bus_late_cleaned.to_excel(output_path, index=False)

In [ ]:
# Save to Excel
output_path = 'bus_accident.xlsx'
bus_late_cleaned.to_excel(output_path, index=False)

In [ ]:
bus_late_cleaned

,노선ID,정류장ID,노드순번,정류장명,정류장번호,노선명,제공건수,오차누계,소요시간누계,평균2분이하건수,...,신뢰도2분초과건수,신뢰도음수오차누계,신뢰도양수오차누계,지수평활2분이하건수,지수평활2분초과건수,지수평활2분이하건수.1,지수평활2분초과건수.1,퓨전2분이하건수,퓨전2분초과건수,season
0,﻿100000017,﻿100000400,﻿5,﻿청와대,﻿01918,﻿TOUR11,﻿100,﻿0,﻿0,﻿100,...,﻿23,﻿49,﻿51,﻿100,﻿0,﻿18,﻿82,﻿100,﻿0,summer
1,﻿100000017,﻿100000401,﻿6,﻿통인시장,﻿01910,﻿TOUR11,﻿133,﻿0,﻿0,﻿133,...,﻿34,﻿50,﻿83,﻿99,﻿34,﻿50,﻿83,﻿133,﻿0,summer
2,﻿100000017,﻿100000402,﻿7,﻿세종문화회관,﻿01911,﻿TOUR11,﻿196,﻿0,﻿0,﻿106,...,﻿88,﻿93,﻿103,﻿102,﻿94,﻿110,﻿86,﻿106,﻿90,summer
3,﻿100000017,﻿100000403,﻿13,﻿인사동.탑골공원,﻿01913,﻿TOUR11,﻿200,﻿0,﻿0,﻿146,...,﻿62,﻿69,﻿131,﻿126,﻿74,﻿38,﻿162,﻿146,﻿54,summer
4,﻿100000017,﻿100000404,﻿14,﻿종묘.세운상가,﻿01914,﻿TOUR11,﻿200,﻿0,﻿0,﻿121,...,﻿61,﻿61,﻿139,﻿126,﻿74,﻿50,﻿150,﻿121,﻿79,summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246701,﻿241461015,﻿232000433,﻿12,﻿아라대교,﻿41007,﻿김포16A,﻿78,﻿0,﻿0,﻿58,...,﻿20,﻿26,﻿52,﻿52,﻿26,﻿1,﻿77,﻿57,﻿21,summer
246702,﻿241461015,﻿232000841,﻿6,﻿대우아파트,﻿35788,﻿김포16A,﻿23,﻿0,﻿0,﻿23,...,﻿0,﻿14,﻿9,﻿23,﻿0,﻿13,﻿10,﻿23,﻿0,summer
246703,﻿241461015,﻿232000842,﻿7,﻿우방아이유쉘,﻿35789,﻿김포16A,﻿27,﻿0,﻿0,﻿27,...,﻿0,﻿14,﻿13,﻿27,﻿0,﻿17,﻿10,﻿27,﻿0,summer
246704,﻿241461015,﻿232000852,﻿4,﻿고촌근린공원,﻿35799,﻿김포16A,﻿14,﻿0,﻿0,﻿14,...,﻿0,﻿8,﻿6,﻿14,﻿0,﻿9,﻿5,﻿14,﻿0,summer


In [ ]:
from google.colab import files
import pandas as pd

print("필요 파일 업로드")
uploaded = files.upload()

필요 파일 업로드


Saving microdust_9.xlsx to microdust_9.xlsx
Saving microdust_1.xlsx to microdust_1.xlsx
Saving microdust_2.xlsx to microdust_2.xlsx
Saving microdust_3.xlsx to microdust_3.xlsx
Saving microdust_4.xlsx to microdust_4.xlsx
Saving microdust_5.xlsx to microdust_5.xlsx
Saving microdust_6.xlsx to microdust_6.xlsx
Saving microdust_7.xlsx to microdust_7.xlsx
Saving microdust_8.xlsx to microdust_8.xlsx
Saving microdust_10.xlsx to microdust_10.xlsx
Saving microdust_11.xlsx to microdust_11.xlsx
Saving microdust_12.xlsx to microdust_12.xlsx


In [ ]:
import pandas as pd

# 1) 각 월별 파일을 변수에 로드
m1  = pd.read_excel('microdust_1.xlsx')
m2  = pd.read_excel('microdust_2.xlsx')
m3  = pd.read_excel('microdust_3.xlsx')
m4  = pd.read_excel('microdust_4.xlsx')
m5  = pd.read_excel('microdust_5.xlsx')
m6  = pd.read_excel('microdust_6.xlsx')
m7  = pd.read_excel('microdust_7.xlsx')
m8  = pd.read_excel('microdust_8.xlsx')
m9  = pd.read_excel('microdust_9.xlsx')
m10 = pd.read_excel('microdust_10.xlsx')
m11 = pd.read_excel('microdust_11.xlsx')
m12 = pd.read_excel('microdust_12.xlsx')

# 2) 변수들을 리스트로 묶기
df_list = [m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12]

# 3) 월별 DataFrame 전처리 & 저장용 리스트
processed = []
for df in df_list:
    # 3-1. '운영기관' 컬럼 삭제
    if '운영기관' in df.columns:
        df = df.drop(columns=['운영기관'])
    # 3-2. '위치' 컬럼이 '서울'인 행만 필터
    df = df[df['위치'] == '서울']
    # 3-3. '측정월' 문자열화 → 월(int) 추출
    df['월'] = df['측정월'].astype(str).str[-2:].astype(int)
    # 3-4. 월별로 계절 매핑
    df['계절'] = df['월'].map(lambda m:
        'spring'   if 3 <= m <= 5 else
        'summer' if 6 <= m <= 8 else
        'fall' if 9 <= m <= 11 else
        'winter'
    )
    # 3-5. 임시 '월' 컬럼 제거
    df = df.drop(columns=['월'])
    # 3-6. 전처리된 DF를 리스트에 추가
    processed.append(df)

# 4) 모든 월 병합
microdust_all = pd.concat(processed, ignore_index=True)

# 5) 결과 확인
print(microdust_all.head())


   지점명   호선  위치     측정월   월평균      계절  평균월평균
0  청량리  1호선  서울  202401  43.1  winter    NaN
1  제기동  1호선  서울  202401  66.5  winter    NaN
2  신설동  1호선  서울  202401  64.7  winter    NaN
3   동묘  1호선  서울  202401  70.7  winter    NaN
4  동대문  1호선  서울  202401  84.7  winter    NaN


In [ ]:
microdust_all

,지점명,호선,위치,측정월,월평균,계절,평균월평균
0,청량리,1호선,서울,202401,43.1,겨울,NaN
1,제기동,1호선,서울,202401,66.5,겨울,NaN
2,신설동,1호선,서울,202401,64.7,겨울,NaN
3,동묘,1호선,서울,202401,70.7,겨울,NaN
4,동대문,1호선,서울,202401,84.7,겨울,NaN
...,...,...,...,...,...,...,...
3978,신림,신림선,서울,202412,20.2,겨울,NaN
3979,서원,신림선,서울,202412,17.7,겨울,NaN
3980,서울대벤처타운,신림선,서울,202412,15.7,겨울,NaN
3981,관악산(서울대),신림선,서울,202412,13.7,겨울,NaN


In [ ]:
import pandas as pd

# 1) df_stats_m9을 엑셀 파일로 저장할 경로 지정
output_path = 'microdust_all.xlsx'

# 2) 엑셀로 저장 (index=False로 인덱스 컬럼 제외)
microdust_all.to_excel(output_path, index=False)

In [ ]:
# 1) m9 데이터프레임을 '지점명-운영기관-호선' 기준으로 그룹화하여 합계·개수·평균 계산
df_stats_m9 = (
    m9
    .groupby(['지점명', '운영기관', '호선','위치'])['월평균']
    .agg(총합='sum', 개수='count')
    .reset_index()
)
# 1-1) 평균월평균 컬럼 추가
df_stats_m9['평균월평균'] = df_stats_m9['총합'] / df_stats_m9['개수']


In [ ]:
from google.colab import files
import pandas as pd

print("필요 파일 업로드")
uploaded = files.upload()

필요 파일 업로드


Saving microdust_9.xlsx to microdust_9.xlsx


In [ ]:
m9  = pd.read_excel('microdust_9.xlsx')

In [ ]:
df_stats_m9

,지점명,운영기관,호선,위치,총합,개수,평균월평균
0,4.19민주묘지,우이신설경전철,우이신설선,서울,813.0,30,27.100000
1,가락시장,서울교통공사,3호선,서울,887.7,30,29.590000
2,가락시장,서울교통공사,8호선,서울,452.4,30,15.080000
3,가산디지털단지,서울교통공사,7호선,서울,624.9,30,20.830000
4,가야,부산보건환경연구원,2호선,부산,498.7,30,16.623333
...,...,...,...,...,...,...,...
675,화정,한국철도공사,일산선,경기,1269.8,30,42.326667
676,회현,서울교통공사,4호선,서울,960.2,30,32.006667
677,효창공원앞,서울교통공사,6호선,서울,625.1,30,20.836667
678,효창공원앞,한국철도공사,경의선,서울,569.3,30,18.976667


In [ ]:
import pandas as pd

# 1) df_stats_m9을 엑셀 파일로 저장할 경로 지정
output_path = 'df_stats_m9.xlsx'

# 2) 엑셀로 저장 (index=False로 인덱스 컬럼 제외)
df_stats_m9.to_excel(output_path, index=False)

print(f"저장 완료: {output_path}")

저장 완료: df_stats_m9.xlsx


# **각종 데이터 전처리 시행착오**

In [ ]:
import pandas as pd
# 파일 경로
metro_confuse_path = "metro_confuse.xlsx"
bus_confuse_path = "bus_confuse.csv"

# 파일 읽기
df_metro_confuse = pd.read_excel(metro_confuse_path)
df_bus_confuse = pd.read_csv(bus_confuse_path)


In [ ]:
# 1단계: 요일구분, 호선, 출발역 기준으로 그룹화해서 평균 계산
df_metro_grouped = df_metro_confuse.groupby(['요일구분', '호선', '출발역'], as_index=False).mean(numeric_only=True)

# 2단계: 시간 컬럼들을 묶어서 시간대별 평균값 계산
time_mapping = {
    '5시': ['5시30분'],
    '6시': ['6시00분', '6시30분'],
    '7시': ['7시00분', '7시30분'],
    '8시': ['8시00분', '8시30분'],
    '9시': ['9시00분', '9시30분'],
    '10시': ['10시00분', '10시30분'],
    '11시': ['11시00분', '11시30분'],
    '12시': ['12시00분', '12시30분'],
    '13시': ['13시00분', '13시30분'],
    '14시': ['14시00분', '14시30분'],
    '15시': ['15시00분', '15시30분'],
    '16시': ['16시00분', '16시30분'],
    '17시': ['17시00분', '17시30분'],
    '18시': ['18시00분', '18시30분'],
    '19시': ['19시00분', '19시30분'],
    '20시': ['20시00분', '20시30분'],
    '21시': ['21시00분', '21시30분'],
    '22시': ['22시00분', '22시30분'],
    '23시': ['23시00분', '23시30분'],
    '00시': ['00시00분', '00시30분'],
}

# 새로운 시간대 컬럼 생성
df_compact = df_metro_grouped[['요일구분', '호선', '출발역']].copy()
for hour, columns in time_mapping.items():
    df_compact[hour] = df_metro_grouped[columns].mean(axis=1)

In [ ]:
# 1) 컬럼 이름 변경: 1시~9시 → 01시~09시
rename_dict = {f"{i}시총승객수": f"{i:02d}시총승객수" for i in range(1, 10)}
df_bus_confuse.rename(columns=rename_dict, inplace=True)

# 2) 정류소명에서 괄호 포함 숫자 제거
df_bus_confuse['정류소명'] = df_bus_confuse['정류소명'].str.replace(r"\s*\(\d+\)", "", regex=True)

# 3) 동일한 (season, 버스번호, 정류소명) 기준으로 평균 계산
df_bus_grouped = df_bus_confuse.groupby(['season', '버스번호', '정류소명'], as_index=False).mean(numeric_only=True)


In [ ]:
# 계절별 일수 정의
season_days = {
    "spring": 92,
    "summer": 92,
    "fall": 91,
    "winter": 90
}

# 모든 시간대 컬럼 추출
time_columns = [col for col in df_bus_grouped.columns if '총승객수' in col]

# 계절별로 해당 일수로 나누기
for season, days in season_days.items():
    mask = df_bus_grouped['season'] == season
    df_bus_grouped.loc[mask, time_columns] = df_bus_grouped.loc[mask, time_columns] / days


In [ ]:
import pandas as pd
# 파일 경로
metro_confuse_path = "metro_confuse.xlsx"
bus_confuse_path = "bus_confuse.csv"

# 파일 읽기
df_metro_confuse = pd.read_excel(metro_confuse_path)
df_bus_confuse = pd.read_csv(bus_confuse_path)


In [ ]:
# 1단계: 요일구분, 호선, 출발역 기준으로 그룹화해서 평균 계산
df_metro_grouped = df_metro_confuse.groupby(['요일구분', '호선', '출발역'], as_index=False).mean(numeric_only=True)

# 2단계: 시간 컬럼들을 묶어서 시간대별 평균값 계산
time_mapping = {
    '5시': ['5시30분'],
    '6시': ['6시00분', '6시30분'],
    '7시': ['7시00분', '7시30분'],
    '8시': ['8시00분', '8시30분'],
    '9시': ['9시00분', '9시30분'],
    '10시': ['10시00분', '10시30분'],
    '11시': ['11시00분', '11시30분'],
    '12시': ['12시00분', '12시30분'],
    '13시': ['13시00분', '13시30분'],
    '14시': ['14시00분', '14시30분'],
    '15시': ['15시00분', '15시30분'],
    '16시': ['16시00분', '16시30분'],
    '17시': ['17시00분', '17시30분'],
    '18시': ['18시00분', '18시30분'],
    '19시': ['19시00분', '19시30분'],
    '20시': ['20시00분', '20시30분'],
    '21시': ['21시00분', '21시30분'],
    '22시': ['22시00분', '22시30분'],
    '23시': ['23시00분', '23시30분'],
    '00시': ['00시00분', '00시30분'],
}

# 새로운 시간대 컬럼 생성
df_compact = df_metro_grouped[['요일구분', '호선', '출발역']].copy()
for hour, columns in time_mapping.items():
    df_compact[hour] = df_metro_grouped[columns].mean(axis=1)

In [ ]:
# 1) 컬럼 이름 변경: 1시~9시 → 01시~09시
rename_dict = {f"{i}시총승객수": f"{i:02d}시총승객수" for i in range(1, 10)}
df_bus_confuse.rename(columns=rename_dict, inplace=True)

# 2) 정류소명에서 괄호 포함 숫자 제거
df_bus_confuse['정류소명'] = df_bus_confuse['정류소명'].str.replace(r"\s*\(\d+\)", "", regex=True)

# 3) 동일한 (season, 버스번호, 정류소명) 기준으로 평균 계산
df_bus_grouped = df_bus_confuse.groupby(['season', '버스번호', '정류소명'], as_index=False).mean(numeric_only=True)


In [ ]:
# 계절별 일수 정의
season_days = {
    "spring": 92,
    "summer": 92,
    "fall": 91,
    "winter": 90
}

# 모든 시간대 컬럼 추출
time_columns = [col for col in df_bus_grouped.columns if '총승객수' in col]

# 계절별로 해당 일수로 나누기
for season, days in season_days.items():
    mask = df_bus_grouped['season'] == season
    df_bus_grouped.loc[mask, time_columns] = df_bus_grouped.loc[mask, time_columns] / days
